In [1]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [2]:
from operator import itemgetter
import logging, sys

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
_DEBUG = False
KAGGLE = False

In [5]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [6]:
# logging.debug('A debug message!')

In [7]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [8]:
l1=[0,1,2,3]
l2=['a','b','c','d']
l3=['hello','how','are','you']

ll=[l1,l2,l3]
# ll[0]=l1
# ll[1]=l2
# ll[2]=l3


list(zip(*ll))

[(0, 'a', 'hello'), (1, 'b', 'how'), (2, 'c', 'are'), (3, 'd', 'you')]

In [9]:
import time
from multiprocessing import Value
from ctypes import py_object

In [10]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [11]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [12]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [13]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [14]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [15]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [16]:
chain_length = 5110101 #1000001

filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

if KAGGLE:
    filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
else:
    filepath_client="data/"
    
# filename_client=f'{filepath_client}sha256_hashchain_client_5M.npy'
filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# Define the number of CPU cores to use
num_cores = 8

In [17]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [19]:
client_seed_array=np.load(filename_client,
                         allow_pickle=True,fix_imports=True,encoding='latin1')
client_seed_array= client_seed_array[:chain_length]
print(len(client_seed_array))
client_seed_array[0:10]

1010102


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76',
       '87fa02b24749184f0ce9183f21715ce556db72ecd1ad4f32b02227b40f3b24ab',
       'edee47aed4d9b598fa647145a17e44f9d155dc53ad231c913ff5c93fc04e56ee',
       'baf9d66be4136fe8acd30c042704b1798380fe0f3ee45d5800ae01db88e03ed3',
       '8eac9ebdcbfcf1720f6b0317fa66569c3129d278c0e2605069e7c5a0a2179c8d',
       'aeab33d31e0fb10105ce7ad75e3a40736db70a824c3ed6489957d6736243184b'],
      dtype='<U64')

In [20]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [21]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [22]:
np.random.seed(5000)
low = 1500
# nonce_array = np.random.randint(low, high=low+chain_length, 
#                                 size=chain_length)

nonce_array = np.arange(low,low+chain_length,1)
np.random.shuffle(nonce_array)
print(len(nonce_array))
print(nonce_array[:10])
pd.Series(nonce_array).nunique()

5110101
[ 638661 1297273 1057802 1410000 2886373 4139846 2635511 4657935 2336816
  640280]


5110101

In [23]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [24]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [25]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [26]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [27]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

#### Top k hash verification of client and server

In [28]:
# # server_hash_orig = "d100f357cf1b05cc23ddc35c5f34a960911f9e67582a6d5ff27d9a210137e9c0"
# # nonce= 2079
# # client_seed_orig = "ppppppd3f19c18078cea0c25643b203c9a56dc87cc5e71fa46c2cf5520ee8477f5b43c"
# # chain_length=100

# # server_hash_orig = "ffed0cf8eb7020128c153bea86b3f29bc328af0451052576211138023bb300bd"
# # nonce= 2078
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# # chain_length=100

# server_hash_orig = "68f1ed13e2f17e3d38fd888e470eaae28720675dc7c5913a6b7fd9586df159b4"
# nonce= 2116
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# client_seed_orig = "woxpwoxpwoxpwoxpwoxp"+"c023a89e8027c289bc936fb9a6be20aa5d1dcba191e70d98004e860fb6c0d5db"
# chain_length=1000



In [29]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [30]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

Function description 

Input
--------
Server Hash
Nonce
Client Seed

Output
----------
Category that has more number of successes - GT or LT <br>
How many number of success ?<br>

Return state of each condition: (Pass, Fail, Tie) <br>
100 values
-----------

1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin

1000 values
----------------
1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin


In [31]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"
def set_tristate(val1,val2):
    if val1 == val2:
        below_state=-1
        above_state =-1
    else:
        below_state = val1 > val2
        above_state = int(~below_state)
        below_state = int(below_state)
    return below_state,above_state

# def set_diffstate(val1,val2):
#     below_state = val1-val2
#     above_state = -1 * below_state
#     return below_state,above_state
def compute_least_condition(bin_counts):
    mask = bin_counts.index < 5
    val_below = bin_counts[mask].min()
    val_above = bin_counts[~mask].min()
    #NOTE: here val_above and val_below are swapped in arguments
    #since least value in below state should be lesser than above state
    below_state,above_state=set_tristate(val_above,val_below)
    return below_state,above_state
def compute_mostlow_condition(bin_counts,low_values_cutoff):
    mask = bin_counts < low_values_cutoff
    mask2 = bin_counts.index < 5
    val_below = bin_counts[mask & mask2].count()
    val_above = bin_counts[mask & ~mask2].count()
    
#     print(f'val_below:{val_below} val_above:{val_above}')
    below_state,above_state=set_tristate(val_below,val_above)
    return below_state,above_state

def compute_conditions(rolls_arr,size,initial_value=None):
    
    rolls_arr = rolls_arr[:size]
    low_values_cutoff = size * 0.1
    
    below_conditions = np.full(5,-100)
    above_conditions = np.full(5,-100)
    #Condition 1: 
    #Total number of low values on the given category is higher than that of other category 
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts_cond1 = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Sum bincounts')
    debugprint(bin_counts_cond1)
#     print(bin_counts.loc[0],bin_counts.loc[1])
    
    below_conditions[0],above_conditions[0]=set_tristate(bin_counts_cond1.loc[1],bin_counts_cond1.loc[0])
    
    bins = list(range(0,10001,1000))
    labels = list(range(0,10))
    bin_counts = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Full bincounts')
    debugprint(bin_counts)
    #Condition 2: 
    #This category has the least value
    below_conditions[1],above_conditions[1]=compute_least_condition(bin_counts)
    
    #Condition 3: 
    #This category has the most number of low values
    below_conditions[2],above_conditions[2]=compute_mostlow_condition(bin_counts,low_values_cutoff)
    
#     print(f'INITIAL roll:  {rolls_arr[0]}')g
    if initial_value is None:
        initial_value=rolls_arr[0]
    initial_bin = get_bin(labels,int(initial_value/1000))
    
    if initial_bin<5:
        mask_filt = bin_counts.index > initial_bin
    else:
        mask_filt = bin_counts.index < initial_bin
    debugprint(f'filtered bincounts with initial bin:{initial_bin}')
    debugprint(bin_counts[mask_filt])
    #Condition 4: 
    #This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
    below_conditions[3],above_conditions[3]=compute_least_condition(bin_counts[mask_filt])
    
    #Condition 5: 
    #This category has the most number of low values
    below_conditions[4],above_conditions[4]=compute_mostlow_condition(bin_counts[mask_filt],
                                                                      low_values_cutoff)
   
    return below_conditions,above_conditions,bin_counts_cond1,bin_counts
    
def predict_seed_output(server_hash,nonce,client_seed,
                        hash_list_server=None,hash_list_nonce=None,
                        initial_value=None):
    chain_length=2000
    if (hash_list_server is None) and (hash_list_nonce is None):
        hash_list_server = generate_hash_chain(server_hash,chain_length)
#     print('Hash chain completed')
    if hash_list_nonce is not None:
        rolls_list  = compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce)
    else:
        rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
#     print('Roll computation completed')
    #100 values conditions
    rolls_arr=np.array(rolls_list)
    cond_100_below,cond_100_above,\
    bin_counts_cond1_100,bin_counts_100 = compute_conditions(rolls_arr,100,initial_value)
    cond_1000_below,cond_1000_above,\
    bin_counts_cond1_1000,bin_counts_1000 = compute_conditions(rolls_arr,1000,initial_value)
    cond_2000_below,cond_2000_above,\
    bin_counts_cond1_2000,bin_counts_2000 = compute_conditions(rolls_arr,2000,initial_value)
#     cond_4000_below,cond_4000_above = compute_conditions(rolls_arr,4000)
    
    cond_below_all = np.concatenate((cond_100_below,cond_1000_below,
                                     cond_2000_below,
#                                      cond_4000_below
                                    ))
    cond_above_all = np.concatenate((cond_100_above,cond_1000_above,
                                     cond_2000_above,
#                                      cond_4000_above
                                    ))
    
    debugprint(cond_below_all)
    debugprint(cond_above_all)
    
    below_passes = (cond_below_all == 1).sum()
    above_passes = (cond_above_all == 1).sum()
    
    if below_passes > above_passes:
        cond_all =  cond_below_all
        passed_state = BELOW
        total_passes = below_passes
    else:
        cond_all = cond_above_all
        passed_state = ABOVE
        total_passes = above_passes
  
    diff = abs(below_passes-above_passes)
    bin_counts_list = [bin_counts_cond1_100,bin_counts_100,
                      bin_counts_cond1_1000,bin_counts_1000,
                      bin_counts_cond1_2000,bin_counts_2000]
    
    if (bin_counts_cond1_100.loc[-1] < 5) & \
    (bin_counts_cond1_1000.loc[-1] < 50) & (bin_counts_cond1_2000.loc[-1] < 100):
        passed_state = INTER
    

    return cond_all,diff,total_passes,passed_state,bin_counts_list



In [32]:
def compare_prev(server_seed_array,
                client_seed,
                nonce_array,value_to_match=None):
#     print('compare_prev STARTED')
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                       client_seed,
                                       nonce_array)
    if value_to_match is None:
        value_to_match = roll_array[0]
    indices =np.where(roll_array==value_to_match)[0]
    indices_prev = indices -1 
#     print(len(indices_prev))
#     print(np.sort(roll_array[indices_prev]))
    prev_roll_match = roll_array[indices_prev]
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts = pd.cut(roll_array, bins,labels=labels).value_counts()
#     print('Sum bincounts')
#     print(bin_counts)
    
    return bin_counts

def validate_test_topk(test,client_seed_data,pass_exp1,pass_exp2,
                       pass_state_reqd=None,last_index=None,result_df=None,
                       client_char_count=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    bincountcols=[]
    bincountprevcols=[]
    sumbincols = ['lt4750','inter','gt5250']
    
    for col in sumbincols:
        bincountprevcols.append('prev_'+str(col))
    bincountprevcols_nonce =['nonce_'+col for col in bincountprevcols]
    for hash_size in [100,1000,2000]:
        for col in sumbincols:
            bincountcols.append(str(hash_size)+'_'+str(col))
        for col in range(10):
            bincountcols.append(str(hash_size)+'_'+str(col))
#     print(bincountcols)

    bincountcols_nonce = ['nonce_'+col for col in bincountcols]
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'total_passes','passed_state',
                                    'total_passes_1','total_passes_2',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] + bincountprevcols+bincountprevcols_nonce +\
                                             bincountcols+bincountcols_nonce)
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
#         if server_count % 10==0:
#             i = len(client_seed_data)-1
#             print(f'server_count:{server_count}')
        
#         if not result_success:
#             i= last_success_index
#             i = len(client_seed_data)-1
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        chain_length=2000
        cur_hash_list = generate_hash_chain(cur_hash,chain_length)
        cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
#         cur_hash_list_rev = generate_hash_chain(cur_hash[::-1],chain_length)
        
#         client_seed_data=np.array(cur_hash_list[1:])
#         client_seed_data=(str(cur_nonce) + pd.Series(client_seed_data)).values
        
##         cur_hash_list=list((cur_hash + pd.Series(np.array(cur_hash_list))).values)
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 1000==0:
                print(f'current scan : {i}')
            
            if client_char_count is None:
                client_seed = client_seed_data[i]
            else:
                client_seed = client_seed_data[i][:client_char_count]

#             client_seed= generate_hash((client_seed_data[i] + prev_match).encode())
#             client_seed= generate_hash((str(cur_nonce)+client_seed_data[i]).encode())
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if (3000<=roll_hash<=7000) or ((i+1) in last_indices):
                total_passes=-1
                passed_state=ABOVE
                passed_state_2=BELOW
 
            else:
                cond_all,diff,total_passes_1,passed_state,bin_counts_list = predict_seed_output(cur_hash,
                                                                              cur_nonce,
                                                                              client_seed,
                                                                             hash_list_server=cur_hash_list)
                #This condition is to reduce time execution
                total_passes_2=0
                passed_state_2=passed_state
                if (pass_exp2>=total_passes_1):
                    cond_all_2,diff_2,total_passes_2,passed_state_2,bin_counts_list_nonce = predict_seed_output(cur_hash,
                                                                                  cur_nonce,
                                                                                  client_seed,
                                                                                 hash_list_nonce=cur_hash_list_nonce)
#                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_nonce_2 = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              hash_list_nonce=cur_hash_list_nonce)
# #                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_rev = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              cur_hash_list_rev)
                total_passes = total_passes_1 + total_passes_2  

            i-=1
#             if (total_passes_1>=pass_exp1) & (total_passes_2>=pass_exp2) & (passed_state==passed_state_2):
#             if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
#                 & (last_index!=i+1):
            if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
                    & (passed_state!=INTER) & ((pass_state_reqd==None) | (pass_state_reqd==passed_state) ):
#                 print(f'Matching client at {i+1} for seed:{cur_seed} nonce:{cur_nonce}')
                match=True
#                 rand = random.randint(1,2)
#                 print(f'random number:{rand}')
#                 if rand==1:
#                     match=True
            elif (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
#             print('Hash Chain starts')
            server_hash_list = generate_hash_chain(cur_hash,10)
            bincount_prev = compare_prev(server_hash_list,client_seed,cur_nonce,value_to_match=roll_hash)
#             print('Nonce Hash Chain starts')
            nonce_hash_list = generate_hash_chain(str(cur_nonce),10)
            bincount_prev_nonce = compare_prev(cur_hash,client_seed,nonce_hash_list,value_to_match=roll_hash)
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            bincount_row =[]
            for bincount_df in bin_counts_list:
                bincount_row.extend(list(bincount_df.values))
#             print(bincount_row)
            bincount_row_nonce =[]
            for bincount_df in bin_counts_list_nonce:
                bincount_row_nonce.extend(list(bincount_df.values))
#             print(bincount_row_nonce)
            result_success = ((passed_state==ABOVE) & (roll_seed_actual>5250))  \
                            | ((passed_state==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,total_passes,passed_state,
                                           total_passes_1,total_passes_2,
                                           roll_seed_actual,roll_hash,roll_blank_server] + \
                                            list(bincount_prev.values) + list(bincount_prev_nonce.values) +\
                                            bincount_row + bincount_row_nonce
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,total_passes,passed_state,roll_seed_actual,roll_hash}')
          
    return result_df
            
    

In [33]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list

def validate_digits(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    feature_chain_length=20,
                    match_digit_indices=None,
                    last_index=None,result_df=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
        
            match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    if not match:
                        print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,pass_state_reqd,
                                           roll_seed_actual,rolls_list]
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df
  

In [34]:

def validate_test_topk_2(test,client_seed_data,hash_count_to_check,pass_state_reqd=BELOW,
                       last_index=None,result_df=None):
    i = len(client_seed_data)-1 #if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index','passed_state',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] )
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        cur_hash_list = generate_hash_chain(cur_hash,hash_count_to_check+1)
#         cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i] 
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hashes=[]
            for hash_chain_val in cur_hash_list:
                roll_hashes.append(calculate_roll(hash_chain_val,client_seed,cur_nonce))
            roll_hashes_arr = np.array(roll_hashes)
            
            roll_hashes_arr_2 = roll_hashes_arr[:len(roll_hashes_arr)-1]

            if pass_state_reqd==BELOW:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]<4750)
                mask = roll_hashes_arr_2<5250
#                 roll_hashes_arr_1stdigit = (roll_hashes_arr_2  / 1000).astype('int')
#                 flag_check_all_1000 = True
#                 for val1000 in range(6,10):
#                     flag_check_all_1000 = flag_check_all_1000 & len(roll_hashes_arr_1stdigit[roll_hashes_arr_1stdigit==val1000])>0
                    
            else:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]>5250)
                mask = roll_hashes_arr_2>4750
            
            match = (len(roll_hashes_arr_2[mask])==0) #& last_rev # & flag_check_all_1000
            roll_hash = roll_hashes_arr[0]
            i-=1
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,BELOW,
                                           roll_seed_actual,roll_hash,roll_blank_server] 
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success,cur_nonce,roll_seed_actual,roll_hashes_arr}')
          
    return result_df
            
    

In [35]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [36]:
# test_file ='/kaggle/input/hashdata/server seed test data 4.csv'
# test_seeds_4=pd.read_csv(test_file)
# print(test_seeds_4.shape)
# test_seeds_4.head()

In [37]:
# test_file ='/kaggle/input/hashdata/server seed test data 5.csv'
# test_seeds_5=pd.read_csv(test_file)
# print(test_seeds_5.shape)
# test_seeds_5.head()

In [38]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [39]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    dictionary_list=[]
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 5)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,
                        mismatch_digit_indices=mismatch_digit_indices)
        
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
#                 result = [result_success,cur_seed, cur_hash, cur_nonce,
#                                                client_seed,client_index,pass_state_reqd,
#                                                roll_seed_actual,rolls_list,rolls_list_nonce]
                

                dictionary_data = {'match': result_success,'seed':cur_seed, 
                                   'hash': cur_hash, 'nonce':cur_nonce,
                                    'client_seed':client_seed,'client_index':client_index,
                                      'passed_state':pass_state_reqd,
                                     'roll_actual':roll_seed_actual,
                                   'roll_hash_list':rolls_list,'roll_hash_list_nonce':rolls_list_nonce
                                  }
                dictionary_list.append(dictionary_data)
                
#                 result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
    
    result_df = pd.DataFrame.from_dict(dictionary_list)

    return result_df


In [40]:
hashval = 'c2367da13ce04a52a496b68488161c5dc7ceb5fbdb924ade1dfe6ae07538c2c8'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 7070 #2390

feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 200 #20

In [41]:
analysis_df=None

In [42]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
start=24
train_data['seed']=np.array(cur_hash_list[start:-1])
train_data['hash']=np.array(cur_hash_list)[start+1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2134


,seed,hash,nonce
0,1be19c264eb8a176f056eb1f6867ce595f62b4766b99714dae7d7cd26c631199,92ab84aab3a3eacb0115458cf12866382b3d47c6bcaa0438c99c66d886d9d504,7070
1,92ab84aab3a3eacb0115458cf12866382b3d47c6bcaa0438c99c66d886d9d504,6070fd65470ad1ad891ac1a8c5227428cdc6a2b3c405797e02d5d63de3986c75,7070
2,6070fd65470ad1ad891ac1a8c5227428cdc6a2b3c405797e02d5d63de3986c75,becc14ddad271d73545edbdbb05b799639a06ad2f8ee8a8344916b7f1a942672,7070
3,becc14ddad271d73545edbdbb05b799639a06ad2f8ee8a8344916b7f1a942672,63b6d0d4d84a82adae5f57f2d1e99e9217b039295c5c5f126a13d23f761e36ed,7070
4,63b6d0d4d84a82adae5f57f2d1e99e9217b039295c5c5f126a13d23f761e36ed,659d8319f54eec9c5821764318442b5bc4ed95cfc1bfc17ea7e6f68259d214a0,7070


In [43]:
PATTERN_99=False
HIGH_ANALYSIS=False

cur_digit = 9
cur_zero_digit = 0

In [44]:
READ_FROM_FILE=False

In [45]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 16 µs, sys: 5 µs, total: 21 µs
Wall time: 22.6 µs


In [46]:
import cProfile as profile
import pstats

In [47]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

    hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)
    
    prof = profile.Profile()
    prof.enable()

    results_df = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=38,
                        last_index=last_index,result_df=results_df,
                        hash_list_nonce=hash_list_nonce)
    prof.disable()
    row = results_df.iloc[len(results_df)-1]
    last_index = row['client_index']

server_count:0
server_count:5
server_count:10
server_count:15
server_count:20
server_count:25
server_count:30
server_count:35
CPU times: user 49 s, sys: 25.4 ms, total: 49 s
Wall time: 49 s


In [48]:
# print profiling output
stats = pstats.Stats(prof).strip_dirs().sort_stats("cumtime")
stats.print_stats(20) # top 10 rows

         73692084 function calls (73691780 primitive calls) in 48.994 seconds

   Ordered by: cumulative time
   List reduced from 430 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.698    1.698   48.994   48.994 3654202707.py:1(generate_data)
  2797162    7.024    0.000   36.277    0.000 2519531764.py:1(calculate_roll)
  2797162    1.266    0.000   19.466    0.000 hmac.py:153(new)
  2797162    9.042    0.000   18.200    0.000 hmac.py:38(__init__)
   934981    9.084    0.000   10.327    0.000 444425644.py:1(predict_digit_pattern)
  2797162    1.382    0.000    8.404    0.000 hmac.py:147(hexdigest)
     7600    0.258    0.000    6.755    0.001 49123795.py:18(compute_multirolls_nonce)
     7600    0.260    0.000    5.992    0.001 49123795.py:12(compute_multirolls)
  2797162    2.148    0.000    5.019    0.000 hmac.py:128(_current)
 11188648    3.182    0.000    3.182    0.000 {method 'update' of '_hashlib.HASH' object

In [51]:
# print profiling output
stats = pstats.Stats(prof).strip_dirs().sort_stats("cumtime")
stats.print_stats(20) # top 10 rows

         98738850 function calls (98404158 primitive calls) in 70.923 seconds

   Ordered by: cumulative time
   List reduced from 575 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.646    1.646   70.932   70.932 429590960.py:1(generate_data)
  2774374    7.154    0.000   36.303    0.000 2519531764.py:1(calculate_roll)
     7601    0.048    0.000   22.303    0.003 indexing.py:705(__setitem__)
     7601    0.045    0.000   21.419    0.003 indexing.py:1553(_setitem_with_indexer)
     7600    0.150    0.000   21.356    0.003 indexing.py:1941(_setitem_with_indexer_missing)
  2774374    1.331    0.000   19.458    0.000 hmac.py:153(new)
     7599    0.184    0.000   19.398    0.003 frame.py:9041(_append)
  2774374    8.968    0.000   18.127    0.000 hmac.py:38(__init__)
   923587    8.838    0.000    9.996    0.000 444425644.py:1(predict_digit_pattern)
  2774374    1.399    0.000    8.325    0.000 hmac.py:147(hexdigest)
 

In [ ]:
# last_index
# results_df['client_index'].min()

In [ ]:
# results_df.head()

In [ ]:
# print(len(results_df))
# results_df['client_seed'].nunique()

In [ ]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
if READ_FROM_FILE:
    results_df = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                 'roll_hash_list_nonce': pd.eval})
else:
    results_df.to_csv(filename,index=False)

In [ ]:
def save_cleaned_results_df(results_df,is_test):
    nonce_suffix = "_nonce"
    rolls_df = results_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    rolls_df_nonce = results_df['roll_hash_list_nonce'].apply(pd.Series)
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df.shape[1])]
    results_df_cleaned = pd.concat([results_df,rolls_df,rolls_df_nonce],axis=1)
    results_df_cleaned=results_df_cleaned.drop(['roll_hash_list','roll_hash_list_nonce'],axis=1)
    if is_test:
        filename = f'data/cleaned_results_df_test_{nonce}_Large_{file_pattern_str}_pattern.csv'
    else:
        filename = f'data/cleaned_results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
    results_df_cleaned.to_csv(filename,index=False)
    return results_df_cleaned

In [ ]:
results_df.head()

In [ ]:
%%time
results_df_cleaned=save_cleaned_results_df(results_df,False)
results_df_cleaned.head()

In [49]:
HIGH_TARGET= 9000 #8944
LOW_TARGET = 1000 #1056
ZERO_COL_CUTOFF = 4

In [50]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

In [51]:
def generate_features(initial_df,istrain):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
#     for col in list(rolls_df.columns)[1:]:
#         rolls_df[f'{col}_dig2'] = ((rolls_df[col] % 1000) / 100).astype('int')
    rolls_df['count_lt_1000'] = (rolls_df < 1000).sum(axis=1)
    rolls_df['count_gt_5000'] = (rolls_df > 5000).sum(axis=1)-1
    
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual']],rolls_df],axis=1)
        mask = create_target_mask(features_df)
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
#         features_df=features_df.drop(['roll_actual'],axis=1)

        features_df['roll_hash_list_nonce']=initial_df['roll_hash_list_nonce']
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = rolls_df
    return features_df


In [ ]:
train_manual = generate_features(results_df[25*train_client_size:50*train_client_size],True)
# train_manual = generate_features(results_df,True)
train_manual.tail()

In [ ]:
cols_roll = [col for col in train_manual.columns if col.startswith('roll_') and col not in ['roll_0','roll_actual','roll_hash_list_nonce']]
print(cols_roll)

In [52]:
def digit_9_stats(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual = (data['roll_actual']>HIGH_TARGET)
        
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual =(data['roll_actual']<LOW_TARGET)
        
    mask_actual = create_target_mask(data,bHighCheck)

        
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [53]:
def add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,ratio_min,
                aft_cur,tr_total,tr_success,
               aft_all,aft_0,aft_1,
               analysis_df):
    if aft_cur >=ratio_min: 
        result=[nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,aft_cur,np.nan,
               aft_all,aft_0,aft_1,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
               tr_total,tr_success,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]
        analysis_df.loc[len(analysis_df.index)]=result  
    return analysis_df

def save_train_stats(data,nonce,zero_col_cutoff,pattern,bHighCheck,ratio_min,
                    analysis_df=None):
    
    digits = [9,8,7,6,5,4,3,2,1]
    for digit in digits:
#         print(f'****** digit : {digit}')
        if analysis_df is None:
            columns = ['nonce','zero_col_cutoff','pattern_99','target_high','digit','cutoff','afterval','tr_ratio','te_ratio',
                      'aft_all','aft_1','aft_2',
                       'pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5','pr_tr_cutoff','pr_val_cutoff',
                      'tr_total','tr_success','te_total','te_success',
                      'te_pr_1_count','te_pr_2_count','te_pr_3_count',
                       'te_pr_4_count','te_pr_5_count',
                      'te_mean_count','te_med_count','te_tr_count','te_val_count']
            analysis_df = pd.DataFrame(columns=columns)

        after_vals = [8,10]
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        mask_actual=create_target_mask(data,bHighCheck)
        for cutoff in cutoff_range:

#             print(f'********** cutoff:{cutoff}')
            mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff) 
            total_all = len(data[mask])
            if total_all ==0:
                success_all=0
                aft_all=0
            else:
                success_all = len(data[mask & mask_actual])
                aft_all =round(100 * success_all/ total_all ,0)
#             print(f'before values - probability: {aft_all} total : {total_all}  success: {success_all}')
            aft = np.zeros(len(after_vals))
            success = np.zeros(len(after_vals))
            total = np.zeros(len(after_vals))

            for i,val in enumerate(after_vals):
                mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
                total[i] = len(data[mask2])
                if total[i] ==0:
                    success[i]=0
                    aft[i]=0
                else:
                    success[i] = len(data[mask2 & mask_actual])
                    aft[i] =round(100 * success[i]/ total[i] ,0)
#                 print(f'AFTER - factor:{val} - probability: {aft[i]} total : {total[i]}  success: {success[i]}')

            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,25,ratio_min,
                                    aft_all,total_all,success_all,
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[0],ratio_min,
                                    aft[0],total[0],success[0],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[1],ratio_min,
                                    aft[1],total[1],success[1],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
    return analysis_df


In [ ]:
# train_manual.head()

In [ ]:
# results_df.iloc[1072]

In [ ]:
train_manual['roll_actual_dig_1'] = (train_manual['roll_actual'] / 1000).astype('int')
print(train_manual['roll_actual_dig_1'].value_counts(normalize=True))

train_manual['count_hi'] = (train_manual[cols_roll] > 5250).sum(axis=1)
train_manual['count_lo'] = (train_manual[cols_roll] < 4750).sum(axis=1)

train_manual['count_dig_1'] = ( (train_manual[cols_roll] / 1000).astype('int')==1).sum(axis=1)
train_manual['count_dig_2'] = ( (train_manual[cols_roll] / 1000).astype('int')==2).sum(axis=1)
train_manual['count_dig_3'] = ( (train_manual[cols_roll] / 1000).astype('int')==3).sum(axis=1)
train_manual['count_dig_4'] = ( (train_manual[cols_roll] / 1000).astype('int')==4).sum(axis=1)
train_manual['count_dig_5'] = ( (train_manual[cols_roll] / 1000).astype('int')==5).sum(axis=1)
train_manual['count_dig_6'] = ( (train_manual[cols_roll] / 1000).astype('int')==6).sum(axis=1)
train_manual['count_dig_9'] = ( (train_manual[cols_roll] / 1000).astype('int')==9).sum(axis=1)
train_manual['count_dig_8'] = ( (train_manual[cols_roll] / 1000).astype('int')==8).sum(axis=1)
train_manual['count_dig_7'] = ( (train_manual[cols_roll] / 1000).astype('int')==7).sum(axis=1)
train_manual['count_dig_0'] = ( (train_manual[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
cols = ['roll_actual','roll_actual_dig_1',
        'count_dig_9','count_dig_0',
        'count_lt_1000','count_gt_5000','roll_0'] + cols_roll

In [ ]:
# for cur_digit in [9,8,7,5]:
#     print()
#     print(f'************************** cur digit {cur_digit} ************************************************')
#     print('************************** HIGH ************************************************')
#     digit_9_stats(train_manual,True)
#     print()
#     print('************************** LOW ************************************************')
#     digit_9_stats(train_manual,False)

In [ ]:
# %%time
# temp_df=None
# for zero_cutoff in range(0,6):
#     temp_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=temp_df)
# temp_df

Prediction

In [ ]:
# # hashval = '393cc340566014a83e78d302f079c98f5cd4e4bc4c34321231cf479559556038'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# seed = 'dummy' # hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# # nonce = 2548 #2390

In [ ]:
# test_data = pd.DataFrame(columns=['seed','hash','nonce'])
# test_data.loc[0,'seed']=seed
# test_data.loc[0,'hash']=hashval
# test_data.loc[0,'nonce'] = nonce

# print(len(test_data))
# test_data.head()

In [ ]:
# if PATTERN_99:
#     match_digit_arr=np.array([9,9])
#     mismatch_digit_arr=np.array([9,9])
# else:
#     match_digit_arr=np.array([0,0])
#     mismatch_digit_arr=np.array([0,0])
    
# match_digit_indices = np.array([1,2])
# match_digit_count_arr=np.array([1])
# mismatch_digit_indices = np.array([0,3])


In [ ]:
# %%time
# # match_digit_arr = np.array(['999',])
# # match_digit_count_arr=np.array([1])
# # match_digit_indices = np.array(['012'])

# # match_digit_arr=np.array([9,9])
# # match_digit_indices = np.array([1,2])
# # match_digit_count_arr=np.array([1])

# if not READ_FROM_FILE_TEST:

#     cur_client_seed_array = client_seed_array_2[:1000001]

#     results_df_test = None

#     last_index_test =  len(cur_client_seed_array)-1 #1024
#     start = 0

#     results_df_test = generate_data(test_data,
#                                 cur_client_seed_array,
#                                  ABOVE,
#                         match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
#                         mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
#                         match_count_exp=10000, #train_client_size,
#                         feature_chain_length=20,
#                         is_data_hash = False,
#                         test_limit=1,
#                         last_index=last_index_test,result_df=results_df_test,
#                         print_client_scan=False)
#     row = results_df_test.iloc[len(results_df_test)-1]
#     last_index_test = row['client_index']

In [ ]:
# print(results_df_test['client_index'].min())
# len(results_df_test)

In [54]:
def generate_test_features(initial_df):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]

    print(cols_roll)
    rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

    rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
    rolls_df['roll_actual'] = initial_df['roll_actual']
    rolls_df['client_seed'] = initial_df['client_seed']
    rolls_df['client_index'] = initial_df['client_index']

    return rolls_df


In [55]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix=""):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                        'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            match_count_exp=10000, #train_client_size,
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
        results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
    test_manual = generate_test_features(results_df_test)
    print('Test Features Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test,test_manual
 

In [57]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test,test_manual=gen_test_data(nonce,hashval,seed,file_suffix="")
print()
print(results_df_test[['seed','hash','nonce']].head(1))
test_manual.head()

seed: 92ab84aab3a3eacb0115458cf12866382b3d47c6bcaa0438c99c66d886d9d504
1
                                                               seed  \
0  92ab84aab3a3eacb0115458cf12866382b3d47c6bcaa0438c99c66d886d9d504   

                                                               hash nonce  
0  c2367da13ce04a52a496b68488161c5dc7ceb5fbdb924ade1dfe6ae07538c2c8  7070  
server_count:0

999999 999999
No further match for seed:92ab84aab3a3eacb0115458cf12866382b3d47c6bcaa0438c99c66d886d9d504 nonce:7070 match count: 8240

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_4

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,7001,4090,6992,5315,8167,7557,6150,8214,8280,4205,4531,1019,2044,2243,6611,5283,5901,9715,188,473,8325,7116,4166,4338,7964,4334,9418,990,9717,4806,7892,952,5151,7060,328,8265,4585,2395,8548,4158,3669,6792,2832,1160,8586,5352,4146,7410,2324,3064,5109,7051,6747,6491,9782,8567,2945,1992,9988,9902,4239,31,26,3,6,2,11,6,6,6,8,7,5,4334,woxpwoxpwoxpwoxpwoxp33ea744970af84690394e365bed7cf2a77b6f143989b071104d806bc57d52ad0,999890
1,6003,6099,3226,8005,1170,9524,3486,2682,1765,5139,6095,8094,8136,7328,6106,3791,3980,4711,7633,4452,6725,1156,9133,4488,3123,2139,131,4920,3226,4020,2995,235,9094,8364,2712,5216,7866,3051,4007,6019,1306,5258,4880,1784,1045,3407,9859,4585,3794,616,858,1385,1064,3,5847,8472,3596,2936,2485,4639,4774,19,36,8,6,10,10,4,5,4,5,3,5,2139,woxpwoxpwoxpwoxpwoxp31c534399d36e2942a285e07710cfa3facf15b94edc81af38826f9d6aef1679f,999860
2,8006,8068,2918,2531,1372,1410,7728,1511,6167,5369,8477,3893,4419,7283,2238,8165,4260,7499,1531,2425,217,8958,2713,2804,6509,5553,6267,7894,335,7789,7266,199,6271,2283,6156,9123,4653,1425,1003,7788,1751,1589,8959,5639,4123,4178,9831,1427,9440,6315,3062,2309,5540,9135,1612,9143,3799,4786,1652,2623,9848,28,31,11,9,3,6,4,6,6,5,7,3,5553,woxpwoxpwoxpwoxpwoxp62ae7430ba50d195d32f6e8364fd4da5d6da97aac21e89f26a785a5e16949a10,999822
3,6009,1818,7736,1125,5364,9187,182,5345,1367,75,3438,7691,9264,6948,5312,7891,900,7526,4501,9670,5272,789,8456,6919,8977,5455,4601,7839,2783,1716,4760,2471,9208,8739,9163,7561,441,7640,7310,5656,5288,1345,3378,4911,5282,5101,732,7298,6386,6378,9834,4012,6876,3996,292,6976,4865,4680,4471,713,5040,32,23,5,2,3,8,10,6,6,3,9,8,5455,woxpwoxpwoxpwoxpwoxpa385c15c33f8d9eb8a3770491a4b5b28753baf84c95cd5c5995db9a323b47ebd,999781
4,4006,5565,5910,1105,8894,2630,5188,2997,7169,6635,1543,6671,5685,8123,7289,39,3671,6571,1892,2086,5794,9051,223,4294,4427,6493,5934,1,1815,7493,1947,7084,1508,2039,8492,3790,8580,2465,6293,7435,3880,8191,6646,5804,3542,4427,5020,9906,5464,1305,7635,9749,4096,6805,715,6839,1051,8348,5145,9285,1854,31,26,9,5,4,4,10,8,4,6,6,4,6493,woxpwoxpwoxpwoxpwoxp37a4e9b19460770dceb2c778f77b026a1ca24229eb9704b72fbf9f2a0ac5c6dc,999593


In [58]:
%%time
READ_FROM_FILE_TEST=False
#Generate hash test data
print(f'{hashval=}')
hash_l2=generate_hash(hashval.encode())
print(f'{hash_l2=}')
results_df_test_hash,test_manual_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
print()
print(results_df_test_hash.head(1))
test_manual_hash.head()

hashval='c2367da13ce04a52a496b68488161c5dc7ceb5fbdb924ade1dfe6ae07538c2c8'
hash_l2='f3f0cf479a6e9d75cf0a3553ba6441813bb7d83213697d9b2d9ea868818a507e'
1
                                                               seed  \
0  c2367da13ce04a52a496b68488161c5dc7ceb5fbdb924ade1dfe6ae07538c2c8   

                                                               hash nonce  
0  f3f0cf479a6e9d75cf0a3553ba6441813bb7d83213697d9b2d9ea868818a507e  7070  
server_count:0

999999 999999
No further match for seed:c2367da13ce04a52a496b68488161c5dc7ceb5fbdb924ade1dfe6ae07538c2c8 nonce:7070 match count: 8052

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,2005,2486,2497,4680,6428,8315,3254,6774,2740,3891,1153,5236,9979,7804,3012,98,9788,2415,1424,9709,1825,3185,6870,1440,1877,2630,6094,2848,7052,2692,7355,4751,8610,624,4719,7532,7851,8344,1682,3766,456,9793,9824,5509,5416,703,9203,9767,3565,1004,3416,8387,7201,1830,1289,3669,1783,9152,6183,735,4110,25,33,10,7,8,4,3,5,8,4,6,5,368,woxpwoxpwoxpwoxpwoxpe806b294fbe0673080f15068b1a30e4f02b5445e917fcd256dde2c7ed1599831,999953
1,1006,1272,7732,5660,6947,2400,2585,7349,8712,1236,4622,5391,3497,9835,4010,9621,4929,6550,151,8800,2048,307,9715,5424,4719,9254,342,4845,5871,4213,7930,3732,5395,8060,3418,2969,6245,7386,9298,9123,6647,3339,1850,3453,5671,5449,8248,2298,3867,1476,7895,3720,5178,3965,4513,5166,5317,2040,9923,204,5485,29,27,4,6,8,7,11,4,7,4,5,4,4685,woxpwoxpwoxpwoxpwoxpffd89ff9f613fe97ba8be62e27cbb3fcd057c687a7fe71562ac81fe9c1ce90f0,999951
2,7008,5532,4832,7737,2910,7182,663,6615,8480,4932,32,4730,5298,6304,7767,8661,3208,2814,7927,5382,6633,801,286,9149,9292,2330,3556,6353,5295,5952,4755,7703,7969,8273,8915,7040,7804,5911,5656,9781,8552,8369,4737,6317,3252,6128,5162,3322,870,5722,2936,7867,8020,6186,282,4672,11,3105,9864,7254,3413,36,20,0,4,6,6,9,7,4,7,10,7,5120,woxpwoxpwoxpwoxpwoxpce0a11c23b2a5e219fc6f66058f53a5a332979abbb6589c5cc4c0ab6a3c38998,999834
3,7005,3016,2851,6613,8852,9209,7720,5528,7733,5694,2580,6293,2654,9536,5503,3199,6309,8469,2380,7937,9057,4869,2364,1617,6435,467,4977,8822,9299,7206,6412,4673,4671,2116,9089,5154,956,869,8782,3472,7135,8159,2492,4686,7790,7401,5759,1572,6274,5343,3929,3629,2962,2470,9114,1009,6821,8705,6510,5834,1975,33,24,4,9,5,5,7,8,6,6,7,3,2986,woxpwoxpwoxpwoxpwoxpa1ffea4b5bafb269a183ea332a856c1e47aef4dc32173f99bf179573a33acfb3,999829
4,2009,3778,3690,8874,7753,3214,5068,2545,8776,5029,9075,3404,7973,4896,1771,180,678,6779,8604,7466,9975,2628,8116,7856,3615,8720,3528,2817,3905,1373,5340,7675,8117,2690,4718,8909,5456,9702,485,2237,9444,3138,9589,9894,8107,8324,2189,2407,6039,1288,457,4869,6071,4063,4075,299,2706,1294,5767,9677,7677,28,28,4,8,8,5,5,3,7,9,6,5,5197,woxpwoxpwoxpwoxpwoxpd0733dc0ad663ea09a32645cc67e27d75dc54076615fa7e8c0f76e350a0f43af,999776


In [ ]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[25]
# cur_hash = cur_hash_list[26]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash25,test_manual_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
# print()
# print(results_df_test_hash25.head(1))
# test_manual_hash25.head()

In [ ]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[24]
# cur_hash = cur_hash_list[25]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash24,test_manual_hash24=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash24")
# print()
# print(results_df_test_hash24.head(1))
# test_manual_hash24.head()

In [ ]:
%%time
results_df_test_cleaned=save_cleaned_results_df(results_df_test,True)
results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash25,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash24,True)

In [ ]:
# %%time
# if PATTERN_99:
#     file_pattern_str = 'x99x'
# else:
#     file_pattern_str = 'x00x'
# filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern.csv'

# if READ_FROM_FILE_TEST:
#     results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
# else:
#     results_df_test.to_csv(filename,index=False)

In [ ]:
# s=70
# e=80
# l=90
# client_seed_min_1=results_df[s*train_client_size:e*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[s*train_client_size:e*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)
# results_1 = results_df[s*train_client_size:e*train_client_size]
# print(results_1['client_index'].describe(percentiles=[0.09,0.1,0.15,0.2,0.25]))

# client_seed_min=results_df[e*train_client_size:l*train_client_size]['client_index'].min()
# client_seed_max=results_df[e*train_client_size:l*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# results_2 = results_df[e*train_client_size:l*train_client_size]
# print(results_2['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [ ]:
# client_seed_min_1=results_df[25*train_client_size:39*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[25*train_client_size:39*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)

# client_seed_min=results_df[39*train_client_size:50*train_client_size]['client_index'].min()
# client_seed_max=results_df[39*train_client_size:50*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# print(results_df[39*train_client_size:50*train_client_size]['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# results_2 = results_df[39*train_client_size:50*train_client_size]
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [ ]:
# mask = (results_df_test['client_index']<=client_seed_max_1) | (results_df_test['client_index']>=client_seed_min_2)
# results_df_test = results_df_test[mask]
# print(len(results_df_test))
# print(results_df_test['client_index'].describe(percentiles=[0.25,0.5,0.52,0.55,0.57,0.6,0.7,0.8,0.9]))

In [ ]:
# test_manual = generate_test_features(results_df_test)

In [ ]:
# # rand=random.randint(0,len(rolls_df)-1)
# rand=np.random.choice(range(0,len(rolls_df)))
# print('random location:',rand)
# rolls_df.iloc[rand]['client_seed']

In [ ]:
# res = []
# for i in range(100):
#     res.append(np.random.choice(range(0,len(rolls_df))) )
    
# pd.Series(np.array(res)).value_counts()

In [59]:

def check_special_pattern_match(data,bHighCheck,cutoff,after_val):
     
#     rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
#     cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'

#     rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

#     rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
#     rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
#     rolls_df['roll_actual'] = initial_df['roll_actual']
#     rolls_df['client_seed'] = initial_df['client_seed']
#     rolls_df['client_index'] = initial_df['client_index']

    
    mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
    return data[mask]

def compareactual(rolls_df,bHighCheck):
    
    mask_actual=create_target_mask(rolls_df,bHighCheck)
    rolls_filt = rolls_df[mask_actual]
    return len(rolls_filt)


def save_test_stats(analysis_df, data,pattern,bHighCheck,is_prod=False):
    
    mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
    analysis_df_filt = analysis_df[mask]
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff
        
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
        data_filt = data[mask]
        total = len(data_filt)
        analysis_df.at[row.Index,'te_total']  = total  
        
        if not is_prod:
            success  = compareactual(data_filt,bHighCheck)
            if total ==0:
                prob=0
            else:
                prob = round(100 * success/total,0)

            analysis_df.at[row.Index,'te_ratio']  = prob  
            analysis_df.at[row.Index,'te_success']  = success  
        
    return analysis_df

        

In [60]:

# bHighCheck = False
# for cur_digit in [9,8,7,5]:
#     for bHighCheck in [True,False]:
#         if bHighCheck:
#             high_low_text = 'HIGH'
#         else:
#             high_low_text = 'LOW'
#         print()
#         print(f'****** cur_digit {cur_digit} ******  {high_low_text} ')

#         for cutoff in range(0,7):
#             rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,25)
#             len_match  = compareactual(rolls_df,bHighCheck)
#             if len(rolls_df) ==0:
#                 success_count=0
#             else:
#                 success_count = round(100 * len_match/len(rolls_df),0)
#             print()
#             print(f'cutoff:{cutoff} General --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')
#             for afterval in [8,10]:
#                 rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,afterval)
#                 len_match  = compareactual(rolls_df,bHighCheck)
#                 if len(rolls_df)==0:
#                     success_count=0
#                 else:
#                     success_count = round(100 * len_match/len(rolls_df),0)
#                 print(f'cutoff:{cutoff} afterval:{afterval} --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')



In [ ]:
# analysis_df_test = save_test_stats(temp_df, test_manual)
# analysis_df_test

In [ ]:
# mask = (analysis_df_test['digit'].isin([1,2,3,4,6])) & (analysis_df_test['tr_ratio']>=13)
# mask2 = mask & (analysis_df_test['te_ratio']>=13)
# print(len(analysis_df_test[mask]))
# print(len(analysis_df_test[mask2]))
# analysis_df_test[mask2]

##### Feature Generation

In [61]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

def generate_features_full(initial_df,istrain,feature_chain_length):
    
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
    rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
    nonce_suffix = '_nonce'
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [62]:
# def generate_roll_features(initial_df,colname,feature_chain_length,
#                            k=4,
#                           suffix=""):
#     rolls_df = initial_df[colname].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}{suffix}' for i in range(rolls_df.shape[1])]
    
#     # roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([str(i) for i in x]))
    
#     cols_roll = [f'roll_{i}{suffix}' for i in range(1,feature_chain_length+1)]
#     print(cols_roll)
    
# #     rolls_df[f'count_lt_1000{suffix}'] = (rolls_df < 1000).sum(axis=1)
# #     rolls_df[f'count_gt_9000{suffix}'] = (rolls_df >= 9000).sum(axis=1)
# #     rolls_df[f'count_gt_5000{suffix}'] = (rolls_df >= 5000).sum(axis=1)
# #     rolls_df[f'count_gt_7000{suffix}'] = (rolls_df >= 7000).sum(axis=1)
# #     rolls_df[f'count_gt_8000{suffix}'] = (rolls_df >= 8000).sum(axis=1)
    
#     rolls_df[f'count_lt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
#     rolls_df[f'count_gt_9000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df[f'count_gt_8000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df[f'count_gt_7000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df[f'count_gt_6000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df[f'count_gt_5000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df[f'count_gt_4000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df[f'count_gt_3000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df[f'count_gt_2000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df[f'count_gt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
#     rolls_df[f'count_hi{suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df[f'count_lo{suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
#     rolls_df[f'count_lo_hi{suffix}'] = rolls_df[f'count_hi{suffix}'] \
#                                             + rolls_df[f'count_lo{suffix}']   

#     first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(1,k+1)]
#     rolls_df[f'count_gt_9000_k{suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    
#     roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([f'{i:04d}' for i in x]))
    
#     rolls_df[f'total_dig_9{suffix}']= roll_list_strs.str.count('9')
#     rolls_df[f'total_dig_0{suffix}']= roll_list_strs.str.count('0')
    
#     rolls_df[f'roll_mean{suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
#     rolls_df[f'roll_std{suffix}'] = (rolls_df[cols_roll]).std(axis=1)
   

# #     if (suffix==""):
# #         rolls_df[f'roll_0_1st_digit'] = (rolls_df['roll_0'] / 1000).astype('int')
# #         rolls_df[f'roll_0_last_digit'] = (rolls_df['roll_0'] % 10)
    
    
#     return rolls_df

# def generate_features_full(initial_df,istrain,feature_chain_length):
    
#     rolls_df=generate_roll_features(initial_df,'roll_hash_list',feature_chain_length,k=4)
#     rolls_df_nonce=generate_roll_features(initial_df,'roll_hash_list_nonce',
#                                     feature_chain_length,k=4,suffix='_nonce' )
    
#     if istrain:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                  rolls_df,
#                                  rolls_df_nonce],axis=1)
#         mask = create_target_mask(features_df)
            
#         features_df['target'] =0
#         features_df.loc[mask,'target'] = 1
#         features_df['roll_actual']=features_df['roll_actual'].astype('int')
#         print(features_df['target'].value_counts())
#     else:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                              rolls_df,rolls_df_nonce],axis=1)
#     return features_df


In [ ]:
# HIGH_ANALYSIS=True

In [63]:
%%time
# train = generate_features_full(results_df[train_client_size:],True,feature_chain_length)
train = generate_features_full(results_df,True,feature_chain_length)
print(len(train))
train.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [64]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [65]:
%%time
test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
print(len(test))
test_hash.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

#### Machine Learning

In [66]:
USE_STAT_FEATS = True

In [67]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in train.columns if col not in all_exclude_cols]
print(len(features))
print(features)

265
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [68]:
X = train[features].drop('target',axis=1)
y = train['target']

In [69]:
X.columns

Index(['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6',
       'roll_7', 'roll_8', 'roll_9',
       ...
       'count_gt_1000_1_60_nonce', 'count_hi_1_60_nonce',
       'count_lo_1_60_nonce', 'count_lo_hi_1_60_nonce',
       'count_gt_9000_k_1_60_nonce', 'count_lt_1000_k_1_60_nonce',
       'total_dig_9_1_60_nonce', 'total_dig_0_1_60_nonce',
       'roll_mean_1_60_nonce', 'roll_std_1_60_nonce'],
      dtype='object', length=264)

In [70]:
# results_df#[['nonce','roll_actual','roll_hash_list','client_seed','client_index']]

In [71]:
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [72]:
n_folds=10
tr_indices_folds = []
val_indices_folds=[]
tr_indices_seeds=[]
for fold in range(n_folds):
#     tr_indices = train[(fold+2) * train_client_size:].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_offset = 1#25
    tr_length = 25#25
    tr_start = (fold+tr_offset) * train_client_size
    tr_end = (fold+tr_offset+tr_length) * train_client_size
    tr_indices = train[tr_start:tr_end].index #fold + 3
#     tr_indices = list(tr_indices) + list(train[70:90].index) #fold + 3
    
#     tr_indices_seed=[]
#     for i in range(1,tr_offset):
#         tr_start_seed = (fold+i) * train_client_size
#         tr_end_seed = (fold+i+1) * train_client_size
#         tr_indices_seed.append(train[tr_start_seed:tr_end_seed].index) #fold + 3 
        
    # tr_start_seed = (fold+1) * train_client_size
    # tr_end_seed = (fold+tr_offset) * train_client_size
    # tr_indices_seed = train[tr_start_seed:tr_end_seed].index #fold + 3
    
    tr_indices_seed = [train[0:(fold+1) * train_client_size].index]
#     val_indices = train[(fold+1) * train_client_size:(fold+2) * train_client_size].index
    val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_indices_seeds.append(tr_indices_seed)
    tr_indices_folds.append(tr_indices)
    val_indices_folds.append(val_indices)
    

In [73]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    
    f1_score = f1_score[:len(thresholds)]
    f1_score[np.isnan(f1_score)]=0
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'{precision=}')
#     print(f'{recall=}')
#     print(f'{f1_score=}')
#     print(f'{thresholds=}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [74]:
ZERO_COL_CUTOFF=4

In [75]:
def train_status(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = 'count_gt_9000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
        cutoff_range = range(2+2,7+2)
        
    else:
        cutoff_col = 'total_dig_0'
        zero_col = 'total_dig_9'
        after_col = 'count_hi'
        cutoff_range = range(2,7)

    mask_actual=create_target_mask(data,bHighCheck)    
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        # print(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+train['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [76]:
def validate_pattern(initial_df,bHighCheck,zero_col_cutoff,digit,cutoff,after_val):
     
    # rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    # rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
    # cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    rolls_df= initial_df
    
    if bHighCheck:
        cutoff_col = f'count_gt_{digit}000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_lt_1000'
        zero_col = f'count_gt_{digit}000'
        after_col = 'count_hi'

    
    # rolls_df['client_seed'] = initial_df['client_seed']
    # rolls_df['client_index'] = initial_df['client_index']

    
    mask = (rolls_df[cutoff_col]==cutoff) \
                            & (rolls_df[after_col]<=after_val)  \
                            & (rolls_df[zero_col]==zero_col_cutoff)   \
                            #& (rolls_df['count_hi']+rolls_df['count_lo']<=18)
    
    return rolls_df[mask]
    

In [77]:
# val_res_df = validate_pattern(X_val,True,4,10)
# print(y_val[val_res_df.index])
# print(len(val_res_df))
# val_res_df

In [ ]:
# probs= xgb_model.predict_proba(val_res_df)[:,1]
# print(probs)
# np.where(probs>0.3)

In [ ]:
# # roll_actual_df = pd.DataFrame(columns=['roll_actual'])
# # roll_actual_df['roll_actual']=train['roll_actual'].iloc[tr_index]
# # print(len(X_tr))
# # print(len(roll_actual_df))

# oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# # fit and apply the transform
# roll_actual_df, y_temp = oversample.fit_resample(train[features+['roll_actual']].iloc[tr_index], 
#                                                  train['target'].iloc[tr_index])
# print(len(roll_actual_df))

# # tr_df = pd.concat([X_tr,roll_actual_df],axis=1)
# # print(len(tr_df))
# train_status(roll_actual_df,True)

In [78]:
%%time
READ_MODEL_FILE = False

if READ_MODEL_FILE:
    xgb_models = joblib.load(f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
else:

    params = { 'n_estimators':100,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              # 'scale_pos_weight': 5,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1}

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]

    for fold,(tr_index, val_index, tr_index_seed) in enumerate(zip(tr_indices_folds,val_indices_folds,tr_indices_seeds)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]

        oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
        # fit and apply the transform
        X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
        print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
        print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')
#         print(f'{tr_index_seed}')
#         print(f'{val_index}')
#         print(f'{tr_index}')

        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_tr,y_tr)
        xgb_models.append(xgb_model)

        mean_opt_cutoff_tr = 0
        for tr_index_seed_cur in tr_index_seed:
            X_tr_seed,y_tr_seed = X.iloc[tr_index_seed_cur],y.iloc[tr_index_seed_cur]

            tr_probs = xgb_model.predict_proba(X_tr_seed)[:,1]
            opt_cutoff_tr, f1score = get_opt_cutoff_prec(y_tr_seed,tr_probs)
            # print('tr cutoff:',opt_cutoff_tr)
            tr_labels = convert_probtolabels(tr_probs,cutoff=opt_cutoff_tr) 
            mask1 = (tr_labels==1)
            mask2 = (y_tr_seed==1)
            total = len(tr_labels[mask1])
            success = len(tr_labels[mask1 & mask2])
            ratio = success/total if total!=0 else 0
            # print(f'tr ratio: {ratio} total: {total} success: {success}')
            mean_opt_cutoff_tr += opt_cutoff_tr / len(tr_index_seed)

        print(f'last cutoff :{opt_cutoff_tr}')
        print(f'mean tr cutoff :{mean_opt_cutoff_tr}')
        tr_last_cutoffs.append(opt_cutoff_tr)
        tr_cutoffs.append(mean_opt_cutoff_tr)
        val_probs = xgb_model.predict_proba(X_val)[:,1]
        print(val_probs[0:5])
        labels = convert_probtolabels(val_probs)
        score_init = accuracy_score(y_val,labels)

        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
        print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        # print(pd.Series(val_probs).describe(percentiles=[0.6,0.7,0.75,0.8]))
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff_tr) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
        print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f} initial accuracy score:{score_init}')
        scores.append(cur_acc)

        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio @val_cutoff: {ratio} total: {total} success: {success}')

    #     train_status(X_tr,True)

    #     top_prob = np.sort(val_probs)[::-1][:1]
    #     top_label = y_val[val_probs==top_prob ]
    #     print(f'top_prob:{top_prob} top_label:{top_label}')

    print(f'mean score:{np.mean(scores)}')    
    print(f'mean ratio:{np.mean(ratios)}')   
    
    print(np.mean(np.array(val_cutoffs)))
    print(np.mean(np.array(tr_cutoffs)))
    print(np.mean(np.array(tr_last_cutoffs)))
    print(ratios)
    
    joblib.dump(xgb_models,f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
    print('Models Save completed')   

0    4519
1    4519
Name: target, dtype: int64

 ******************* fold: 0 ********
tr size: 9038  val size:  200
[07:22:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
last cutoff :0.2769646942615509
mean tr cutoff :0.2769646942615509
[0.4292892  0.18314786 0.4546743  0.26659068 0.2874474 ]
valid cutoff: 0.2769647
ratio: 0.13333333333333333 total: 105 success: 14
val accuracy score:0.49 f1 score:0.4188 initial accuracy score:0.835
ratio @val_cutoff: 0.13333333333333333 total: 105 success: 14
0    4517
1    4517
Name: target, dtype: int64

 ******************* fold: 1 ********
tr size: 9034  val size:  200
[07:22:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:111

NameError: name 'file_pattern_str' is not defined

##### Prediction of Test 

In [ ]:
# results_df[:train_client_size].head()

In [ ]:
# results_df_test.head()

In [ ]:
# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     for row in analysis_df.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for model_ind in model:
#                 probs+= model_ind.predict_proba(test_filt_df[features_test])[:,1] / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#         prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]         
#     return analysis_df


In [ ]:
# def save_test_ml_probs_internal(analysis_df_filt,
# #                                 features_test,
# #                                 test,model,
#                                 prob_tr_cutoff,prob_val_cutoff,
#                                 pattern,bHighCheck,multi_models):
    
#     start = time.time()
#     #get data from shared memory
#     features_test = p.value[0]
#     test = p.value[1]
#     model = p.value[2]
# #     print(len(test))
# #     print(test.head())
#     end = time.time()
#     print(f'Read Shared memory time:{end-start}')
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df_filt.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df_filt.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df_filt.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df_filt.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df_filt.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df_filt.at[row.Index,'pr_val_cutoff']  = ratios[8]       

#     end = time.time()
#     print(f'Internal function time:{end-start}')
#     return analysis_df_filt  

# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
#     other_args=[#features_test,test,model,
#                 prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,multi_models]
#     analysis_df_filt=parallelize_dataframe(analysis_df_filt, save_test_ml_probs_internal, 
#                                            data_to_share=[features_test,test,model],
#                                            other_args=other_args, n_cores= multiprocessing.cpu_count())
#     if len(analysis_df_filt)==len(analysis_df):
#         analysis_df=analysis_df_filt
#     else:
#         analysis_df=pd.concat([analysis_df[~mask],analysis_filt],axis=0)
#     return analysis_df

In [ ]:
# temp_df = pd.DataFrame()
# temp_df['one'] = np.array([3,42,20,8,19,54,72,15])
# temp_df['two'] = np.array([0,1,3,1,1,2,2,3])

# temp_df_filt = temp_df[temp_df['one']>=42]
# for row in temp_df_filt.itertuples():
#     print(row.Index)
# #     row.loc[row.Index,'two']=5
#     temp_df.loc[row.Index,'two']=5
# print(temp_df_filt.head())    
# temp_df

In [ ]:
# def save_test_ml_probs_old(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]  
        
#     end = time.time()
#     print(f'Function time:{end-start}')   

#     return analysis_df

In [79]:
MEAN_CUTOFF=-1
MEDIAN_CUTOFF=-2
TR_CUTOFF=-3
VAL_CUTOFF=-4


def save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                      analysis_df,row_index,
                      bHighCheck):
    probs = test_filt_df['probs']
    prob_cutoffs = [np.mean(probs),
#                     np.percentile(probs,0.75),
                    np.median(probs),
                    0.1,0.2,0.3,0.4,0.5,
                    prob_tr_cutoff,prob_val_cutoff]
    ratios = np.zeros(len(prob_cutoffs))
    total = np.zeros(len(prob_cutoffs))
    for i,prob_cutoff in enumerate(prob_cutoffs):
        test_filt_prob=test_filt_df.loc[probs>prob_cutoff]
#         indices= np.where(probs>prob_cutoff)[0]
#         roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
        total[i] = len(test_filt_prob)
        mask=create_target_mask(test_filt_prob,bHighCheck)
        success = len(test_filt_prob[mask])
        if total[i]==0:
            ratios[i]=0
        else:
            ratios[i] = 100 * success / total[i]

        if (i >= 2) & (i<=6):
            analysis_df.at[row_index,f'pr_{i-1}']  = ratios[i] 
            analysis_df.at[row_index,f'te_pr_{i-1}_count']  = total[i] 

#             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')

    analysis_df.at[row_index,'pr_mean']  = ratios[0]  
    analysis_df.at[row_index,'pr_med']  = ratios[1] 
    analysis_df.at[row_index,'pr_tr_cutoff']  = ratios[7]  
    analysis_df.at[row_index,'pr_val_cutoff']  = ratios[8]  
    
    analysis_df.at[row_index,'te_mean_count']  = total[0]  
    analysis_df.at[row_index,'te_med_count']  = total[1] 
    analysis_df.at[row_index,'te_tr_count']  = total[7]  
    analysis_df.at[row_index,'te_val_count']  = total[8]
    
    return analysis_df


def map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff):
    
    if prob_cutoff==MEAN_CUTOFF:
        prob_cutoff=np.mean(probs)
    elif prob_cutoff==MEDIAN_CUTOFF:
        prob_cutoff=np.median(probs)
    elif prob_cutoff==TR_CUTOFF:
        prob_cutoff=prob_tr_cutoff
    elif prob_cutoff==VAL_CUTOFF:
        prob_cutoff=prob_val_cutoff
        
    return prob_cutoff

def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

def save_test_ml_probs(analysis_df,test,model,
                       prob_tr_cutoff,prob_val_cutoff,
                       pattern,bHighCheck,
                       gen_probs=True,
                       is_prod=False,
                       prob_cutoff=-1,
                      multi_models=False):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
#         print(f'{filt_size=}')
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue
        if gen_probs:
            test_filt_df = generate_probs(multi_models,model,test_filt_df,features_test)
        probs = test_filt_df['probs'].values
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]

        if is_prod:
            prob_cutoff=map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff)
            mask = (test_filt_df['probs'] > prob_cutoff)
            test_filt_df=test_filt_df[mask]
        
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
        if not is_prod:
            analysis_df = save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                              analysis_df,row.Index,bHighCheck)
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all,analysis_df

In [ ]:
# features_test = [col for col in features if col!='target']
# test_filt_df = validate_pattern(test[features_test],True,4,9,0,8)

# roll_filt = test['roll_actual'].loc[test_filt_df.index]
# total = len(roll_filt)
# if HIGH_ANALYSIS:
#     success = len(roll_filt[roll_filt>=9000])
# else:
#     success = len(roll_filt[roll_filt<1000])
# if total==0:
#     ratio=0
# else:
#     ratio = success / total
# print(f'ratio:{ratio} total:{total} success:{success}')
# # test_filt_df

In [ ]:
# probs= xgb_model.predict_proba(test_filt_df)[:,1]
# # print(probs)
# print(np.mean(probs))
# print(np.median(probs))
# prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5]

# for prob_cutoff in prob_cutoffs:
#     indices= np.where(probs>prob_cutoff)[0]
# #     print(len(indices))
#     # print(indices)
#     # print(test_filt_df.iloc[indices].index)
#     roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#     total = len(roll_filt)
#     if HIGH_ANALYSIS:
#         mask = roll_filt>=9000
#         mask_arr =roll_filt.values>=9000
#     else:
#         mask=roll_filt<1000
#         mask_arr =roll_filt.values<1000
# #     print(roll_filt.tail())
#     success = len(roll_filt[mask])
# #     print(np.where(mask_arr))
#     if total==0:
#         ratio=0
#     else:
#         ratio = success / total
#     print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')


In [80]:
CONT_ANALYSIS=False
IS_PROD=False

In [81]:
if not IS_PROD:
    
    actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
#     actual_seed = cur_hash_list[25]
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(actual_seed,
                                           results_df_test['client_seed'],
                                           nonce)

    results_df_test['roll_actual']=roll_array
    test_manual = generate_test_features(results_df_test)
    test = generate_features_full(results_df_test,False,feature_chain_length)

b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 

In [82]:
l1=list(range(1,261))
# l2=list(range(50,80))
for hash_idx in [-1]+l1:
    
    if hash_idx==-1:
        actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
        suffix = 'orig'
    else:    
        actual_seed = cur_hash_list[hash_idx]
        suffix = hash_idx
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    test[f'roll_actual_{suffix}']=vectorized_calculate_roll(actual_seed,
                                           test['client_seed'],
                                           nonce)

b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263
f3f0cf479a6e9d75cf0a3553ba6441813bb7d83213697d9b2d9ea868818a507e
95fa727186aff5492e2b91ed18ec5f2d70294114cb30027912dc21dec25a1293
baf6f90cbeb9f7679425c481887f4d9c43a352c83ab2279beb772f8a42117dcb
ca2a3eab3ff81f01bfd16385728223e3fe96b7b727ac81e3d0cde307c0471ee0
eadaa2fea4de1c73d59eeb7e6509543ff6b1a7f858d76e07aada67c8aa1c6660
065f6ea6a208b32b936291e438c8526843a33f3c5cbf5ca465d951fa52e34867
3e54193f7ead053fd7d694a5397d49185d477f43b3c5a6c5bbec87078fe797d0
3f3dadc04d9573bb2da95e50a631dd831510abb7b53f07f9fe9a22508ff779aa
b59ce975aedd64328423c0b5f2818ea88ea4fa6efe18c7e28907bb40c8634fd0
2555b016817982ecead038b33a5257e279b4915423c444dc18f987107a01c1db
068ca65afe928201932a4d4eec13a0489c181157cfda87568096edeec52729d4
94d45ed8d01120ddd51bc0e335cdda89e9d5dec0e4b77582ac8804c2cf06048e
3a025a0c7255ebb6da8fbbc062acadfa2135c4ccc03172f97a8b743bd30d7de6
25bcc5a258729509cc9341b8e1160bf1bdd6b602beea6494dde4296d22e00782
a1df1ca8a91269a79e3c02e7a

e75f7296722d1566e435f157068be1da068656023a66da9253c144407d5f169c
c18ba33728f2b244feb9c96f68e807ffb45f965da3ab9796f336926b6fb16b0f
144af181eb4bd43369e3154e52ab910d76a135dc9cecc4bf97651237fecb2186
9a130fc10c4e2166a03c786d01d4c0ea00eb9999e57b20276d1139e9ca810929
28c6c17124e9f2b7cbfcea449df1afd00ad00b89ae2ba18823848bc4ae951f77
67bc851355d72636390bd47c335144c610d82eba75287c5cce408dcb6cc8bc84
614501288957b2412cbbec54e3e58cc30e4a414d2813e54a35f24d2349149652
ca57ac4c4d41caa5d7afd200aed3ad65591abe1e93728cb2542ebde454c8a855
0b95ae0eb0119e02b4f4bcd88cf98a4445761aca9e657cad7f8c01cbb5d4afe7
60d664eea03fcfa9a836b2eec196e79ac2b0698dfdd88aa16b5d5801ba77ce59
d71f3166465f81141f4736d8dadaeb9cbf46b46b0b73b4729f6ac67a9d39ff26
b2d7f5cb84989391f9680dd37d8dd09e12da8194e7a2698e62f854f6ed10a999
8bdefcec0f655f26d72b0f82beccd4658eb0250bf2310630ca30b68e2418c3da
d2ff1a81931de5b1302ee82e994331132a6d2d0e36951a5f224e88553866e9ea
9211dd6bd43ea2750df3b6a9b7ce309bcd5195c13ff98f0bc26da86a84099f79
11132225600d763b749855857

a185e1c642c92f41c19cf72eb35eaa843e165d78897b9594f8578d539f333911
2ea03b3a6afba157aa394df5557d5e147685e15ecff3b23af18e7b0199497a89
4a1063f304acadfa1944b0b6dc91ca9f57d21b8dfd4b28c4f7339ddf4df3f01d
d6bfd67b6c45f68d4c794fd57dc36a32d5fbdc9066b6788fa3480ffcb377a581
1fcfb646cb7530f900ca161681433693702f7d52733cbffe80481282a8c53fe2


In [ ]:
# analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')

In [83]:
mean_imp = np.zeros(len(features)-1)
imp_df = pd.DataFrame()
imp_df['feature'] = [col for col in features if col!='target']
for i,model in enumerate(xgb_models):
    cur_imp = model.feature_importances_
    imp_df[f'importance_m{i}'] = cur_imp
#     print(cur_imp)
    mean_imp += cur_imp / len(xgb_models)

# print(mean_imp)
imp_df['importance'] = mean_imp
imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
imp_df

,feature,importance_m0,importance_m1,importance_m2,importance_m3,importance_m4,importance_m5,importance_m6,importance_m7,importance_m8,importance_m9,importance
0,roll_mean_1_60_nonce,0.005625,0.004401,0.006040,0.005261,0.006572,0.006735,0.004254,0.003607,0.005277,0.004235,0.005201
1,roll_32_nonce,0.004356,0.004634,0.006058,0.005059,0.005722,0.005635,0.005369,0.005289,0.004617,0.004842,0.005158
2,roll_std_50_60,0.005830,0.004945,0.005799,0.005599,0.005315,0.004966,0.004270,0.005666,0.004918,0.002954,0.005026
3,count_gt_1000_1_60_nonce,0.004878,0.005762,0.004453,0.007990,0.003840,0.003305,0.003796,0.006334,0.003831,0.005453,0.004964
4,roll_45,0.004281,0.006895,0.006090,0.004262,0.003536,0.003835,0.004959,0.003846,0.005834,0.005063,0.004860
5,roll_51_nonce,0.004727,0.004483,0.004948,0.005207,0.004631,0.004497,0.005408,0.005738,0.003701,0.004831,0.004817
6,roll_33_nonce,0.004740,0.005993,0.004703,0.004280,0.005627,0.004983,0.004111,0.004874,0.005118,0.003703,0.004813
7,roll_46_nonce,0.004150,0.004083,0.004478,0.004157,0.005175,0.004894,0.005360,0.005636,0.005077,0.005119,0.004813
8,count_gt_9000_k_25_50,0.003319,0.006382,0.001220,0.003681,0.005592,0.010208,0.005819,0.002845,0.006309,0.002465,0.004784
9,roll_34_nonce,0.004935,0.004026,0.005064,0.004866,0.006081,0.004701,0.004517,0.004947,0.004787,0.003873,0.004780


In [ ]:
# %%time
# test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
# test_hash25.head()

In [ ]:
# %%time
# test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
# test_hash24.head()

In [84]:
test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [85]:
test_filt=test.copy()
test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [86]:
test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
cutoffs = []
ratios =[]
totals =[]
print(test_hash_probs['probs'].describe())
cutoffs = np.linspace(test_hash_probs['probs'].min(),
                      test_hash_probs['probs'].quantile(0.95),700)
for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
#     cutoff=0.4
#     diff = 0.01
#     print(cutoff,cutoffs[i+1])
    mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
    mask2=create_target_mask(test_hash_probs)
    
#     mask2 = test_hash_probs['roll_actual']>=9000
    total = len(test_hash_probs[mask])
    if total==0:
        ratio=0
        success=0
    else:
        success = len(test_hash_probs[mask & mask2])
        ratio = success / total
    ratios.append(ratio)
    totals.append(total)
#     print(ratio,success,total)

print(len(ratios),len(cutoffs))
df=pd.DataFrame()
df['cutoff']=cutoffs[:len(cutoffs)-1]
df['cutoff_2']=cutoffs[1:len(cutoffs)]
df['ratio'] =ratios
df['total'] =totals

df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
print(df[:5].mean())
print(df[:10].mean())
df.head(10)

count    8052.000000
mean        0.294464
std         0.079208
min         0.067006
25%         0.237282
50%         0.291676
75%         0.349459
max         0.564358
Name: probs, dtype: float64
699 700
cutoff      0.118569
cutoff_2    0.119086
ratio       0.700000
total       1.600000
dtype: float64
cutoff      0.187027
cutoff_2    0.187544
ratio       0.550992
total       4.500000
dtype: float64


,cutoff,cutoff_2,ratio,total
0,0.122806,0.123323,1.000000,1
1,0.112473,0.112989,1.000000,1
2,0.133656,0.134173,0.500000,2
3,0.109889,0.110406,0.500000,2
4,0.114023,0.114539,0.500000,2
5,0.374938,0.375454,0.444444,9
6,0.399221,0.399737,0.428571,7
7,0.171372,0.171889,0.428571,7
8,0.185322,0.185839,0.375000,8
9,0.146573,0.147089,0.333333,6


In [87]:
df

,cutoff,cutoff_2,ratio,total
0,0.122806,0.123323,1.0,1
1,0.112473,0.112989,1.0,1
2,0.133656,0.134173,0.5,2
3,0.109889,0.110406,0.5,2
4,0.114023,0.114539,0.5,2
...,...,...,...,...
694,0.158456,0.158972,0.0,4
695,0.157939,0.158456,0.0,7
696,0.156906,0.157422,0.0,5
697,0.156389,0.156906,0.0,3


In [88]:
df.describe()

,cutoff,cutoff_2,ratio,total
count,699.000000,699.000000,699.000000,699.000000
mean,0.247322,0.247839,0.082382,10.942775
std,0.104329,0.104329,0.106096,7.487880
min,0.067006,0.067523,0.000000,0.000000
25%,0.157164,0.157681,0.000000,4.000000
50%,0.247322,0.247839,0.058824,11.000000
75%,0.337480,0.337996,0.142857,17.000000
max,0.427637,0.428154,1.000000,30.000000


In [89]:
print(len(df[df['total']==1]))
print(len(df[(df['total']==1) & (df['ratio']>0)]))
df[df['total']==1]

27
2


,cutoff,cutoff_2,ratio,total
0,0.122806,0.123323,1.0,1
1,0.112473,0.112989,1.0,1
458,0.067006,0.067523,0.0,1
464,0.111439,0.111956,0.0,1
465,0.110923,0.111439,0.0,1
466,0.110406,0.110923,0.0,1
473,0.106273,0.106789,0.0,1
474,0.105756,0.106273,0.0,1
477,0.104206,0.104723,0.0,1
482,0.101623,0.102140,0.0,1


In [90]:
df[df['ratio']<0.12].index[0]

217

In [91]:
test['roll_actual']=test[f'roll_actual_orig'].copy()

In [92]:
test_probs = generate_probs(True,xgb_models,test,features_test)
top_k=[]
all_k=[]
for k in range(len(df)):
    row = df.iloc[k]
#     print(row['ratio'],row['cutoff'],row['cutoff_2'])
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    mask2 = create_target_mask(test_probs)    
    test_filt = test_probs[mask]

    total = len(test_filt)
    if total==0:
        success=0
        ratio=0
    else:
        success = len(test_probs[mask & mask2])
        ratio = success / total
    if total==1:
        all_k.append(k) 
    if ratio>=0.12:
        if total==1:
            top_k.append((k,ratio,total))
#     print(ratio,success,total,row['cutoff'],k)
print(len(all_k))
print(all_k)
print(len(top_k))
print(top_k)
mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
test_filt_hash  = test_hash_probs[mask]

31
[15, 16, 17, 44, 152, 463, 466, 471, 472, 477, 478, 484, 485, 487, 495, 500, 501, 504, 512, 514, 515, 550, 558, 559, 561, 563, 573, 577, 650, 659, 660]
1
[(487, 1.0, 1)]


In [93]:
all_k_arr = np.array(all_k)
diff = np.diff(all_k_arr)
print(diff)
ind = np.where(diff>=20)
print(ind[0]+1)
trans_k = np.sort(np.concatenate((all_k_arr[ind[0]],all_k_arr[ind[0]+1])))
print(len(trans_k))
trans_k

[  1   1  27 108 311   3   5   1   5   1   6   1   2   8   5   1   3   8
   2   1  35   8   1   2   2  10   4  73   9   1]
[ 3  4  5 21 28]
10


array([ 17,  44,  44, 152, 152, 463, 515, 550, 577, 650])

In [110]:
def get_sim_cutoff_data(test,xgb_models,features_test,test_hash_cutoff_df,
                       is_imp_feats,imp_feats_count,gen_probs=True):
    if gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test)
    else:
        test_probs = test
    print('Generate probs completed')
#     test_probs['roll_actual_init']=test_probs['roll_actual'].copy()
    
    roll_actual_init = test_probs['roll_actual'].to_numpy()

    if is_imp_feats:
        hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
#         hash_groups = [['roll_mean_nonce']]
    else:
    #     hash_groups = [['roll_21_nonce', 'roll_37_nonce', 'roll_mean_nonce','roll_36_nonce','roll_48',
    #                    'roll_53_nonce','roll_32_nonce','roll_44_nonce','roll_38_nonce','roll_17']]
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
#         hash_groups = [list(range(25,36)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
    print(hash_groups)
    print('list creation complete')
    

    top_k=[]
#     sim_cutoff_df = pd.DataFrame(columns=['k','ratio','total','hash_no','hash_group','roll_mean'])
    df_size = len(test_hash_cutoff_df)
    dictionary_list=[]
    for i,row in enumerate(test_hash_cutoff_df.itertuples()):
        k=row.Index
#         if (k%10)==0:
#             print(f'loop for k = {k}')
#         row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row.cutoff) & (test_probs['probs']<=row.cutoff_2)
        test_filt = test_probs[mask]
        total = len(test_filt)
        ratios=[]

        for idx in hash_groups_flat:
#             print(f'k:{k} idx:{idx}')
            if is_imp_feats:
                test_filt['roll_actual']=test_filt[idx]   
            else:
                test_filt['roll_actual']=test_filt[f'roll_actual_{idx}']
            mask2 = create_target_mask(test_filt)
            if total==0:
                success=0
                ratio=0
            else:
                success = len(test_filt[mask2])
                ratio = success / total
#             print(ratio,success,total,row['cutoff'],k)
            roll_mean=test_filt['roll_actual'].mean()
#             sim_cutoff_df.loc[len(sim_cutoff_df.index)] = [k,ratio, total, idx, np.nan,roll_mean]
        
            dictionary_data = {'k': k,'ratio':ratio, 
                       'total': total, 'hash_no':idx,
                        'hash_group':np.nan,
                          'roll_mean':roll_mean,
                      }
            dictionary_list.append(dictionary_data)
                
    
    sim_cutoff_df = pd.DataFrame.from_dict(dictionary_list)

    for group_no,group in enumerate(hash_groups):
        start =group[0]
        end = group[-1]
        mask = (sim_cutoff_df['hash_no']>=start) & (sim_cutoff_df['hash_no']<=end) 
        sim_cutoff_df.loc[mask,'hash_group']=group_no
        
#     test_probs['roll_actual']=test_probs['roll_actual_init'].copy()
    test_probs['roll_actual']=roll_actual_init
    
    return sim_cutoff_df,test_probs
        



In [95]:
def get_hash_prob_top_data(test,xgb_models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = None,is_min_total=True,
                           total_cri=12,top_n=10):

    if sim_cutoff_df is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test,xgb_models,features_test,
                                                      test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count)
    else:
        test_probs = generate_probs(True,xgb_models,test,features_test)
        
    print(len(sim_cutoff_df))
#     print(sim_cutoff_df.head())
    #filter only high total records
    if is_min_total:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']>=total_cri]
    else:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']<=total_cri]
        if total_cri!=1:
            sim_cutoff_df_filt = sim_cutoff_df_filt[sim_cutoff_df_filt['total']>1]
    print(len(sim_cutoff_df_filt))
    print(sim_cutoff_df_filt.dtypes)
    sim_cutoff_df_filt['total']=  sim_cutoff_df_filt['total'].astype('float')
    sim_summary  = sim_cutoff_df_filt.groupby(['k']).agg( total =('total','mean'),
                                                        roll_mean=('roll_mean','mean'),
                                                         ratio_mean=('ratio', 'mean'), 
                                                          ratio_std=('ratio', 'std')).reset_index()
#     print(sim_summary.head(20))
#     print(sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=False))
    
    top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[False,True])[:top_n].reset_index(drop=True)
    print('--- Top 5 data ---')
    print(top_mean_data[:5])
    print('--- Top 6 to 10 data ---')
    print(top_mean_data[5:])
#     top_std_row = top_mean_data.sort_values('ratio_std',ascending=True).iloc[0]
#     print('--- Selected K ---')
#     print(top_std_row)
#     k_sel = int(top_std_row['k'])

#     k_sel = int(top_mean_data[0:1]['k'])
#     row = test_hash_cutoff_df.iloc[k_sel]
#     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
#     test_sel = test_probs[mask]
#     print('Test Size:',len(test_sel))
    
    return top_mean_data


In [96]:
# k_sel,test_sel = get_test_sel_hash_prob(test,xgb_models,features_test,df,
#                           min_total=12, top_n=10)
# print(k_sel)

In [97]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)]):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        print()
        print(f'********************** Feature count {imp_feats_count} *****************')
        if gen_sim_cutoff:
            sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count,gen_probs)
            sim_cutoff_dfs.append(sim_cutoff_df)
        else:
            sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
            print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
            top_mean_data = get_hash_prob_top_data(test,models,features_test,
                                       test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                                       sim_cutoff_df = sim_cutoff_df,
                                        is_min_total=condn[0], total_cri=condn[1],
                                        top_n=condn[2])
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,test_probs


In [98]:
import math

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [99]:
# k_sel_list=[218,198,201,190,184]
# print(df[df.index.isin(k_sel_list)])
# test_temp=None
# for k_sel in k_sel_list:
#     row = df.loc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_temp is None:
#         test_temp = cur_test_sel
#     else:
#         test_temp = pd.concat([test_temp,cur_test_sel],axis=0)

# cols = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:20]        
# # test_temp
# test_temp['feats_mean']=test_temp[cols].mean(axis=1)
# test_temp['feats_std']=test_temp[cols].std(axis=1)
# test_temp['feats_count']=(test_temp[cols]<1000).sum(axis=1)
# test_temp[['k','roll_actual','feats_mean','feats_std','feats_count']+cols]

In [ ]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [100]:
count_all_k = len(all_k)
if count_all_k<10:
    print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')
count_top_k = len(top_k)    
if count_top_k<2:
    print(f'PROJECT WARNING: number of probable success is {count_top_k} which is less than 2')  
hash_success = len(df[(df['total']==2) & (df['ratio']>0)])    
if hash_success<2:
    print(f'PROJECT WARNING: number of hash success is {hash_success} which is less than 2')

PROJECT WARNING: number of probable success is 1 which is less than 2


In [101]:
print(len(test))
mask = test.duplicated(subset='client_seed')
print(len(test[mask]))
print(test[mask]['client_seed'].nunique())

print(len(test_hash))
mask = test_hash.duplicated(subset='client_seed')
print(len(test_hash[mask]))
print(test_hash[mask]['client_seed'].nunique())

8240
0
0
8052
0
0


In [ ]:
# test=test.drop_duplicates(subset='client_seed')
# test_hash=test_hash.drop_duplicates(subset='client_seed')

In [ ]:
# %%time
# sim_cutoff_dfs_temp=None
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# cutoff_pos_df_temp,sim_cutoff_dfs_temp,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin([2,22,70,100,178,200,486,
#                                                     488,525,604,672,681,747,
#                                                    812,1212,1392,1536])],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_temp,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_temp

In [ ]:
# %%time
# imp_feats_count_list = [30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[df['total']<=10],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,10,20),(False,2,20)])
# cutoff_pos_df_hash

In [ ]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [ ]:
# print(len(df))
# df_filt = df[(df['total']>=50)]
# print(len(df_filt))
# # df_filt[df_filt['ratio']==0.12]

In [102]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30]
sim_cutoff_dfs=None
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']==1)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_hash

max_good_k=457

********************** Feature count 2 *****************
Generate probs completed
[['roll_mean_1_60_nonce', 'roll_32_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 2 *****************
54
54
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  531    1.0  2638.958333         0.5   0.707107
1  488    1.0  3129.550000         0.0   0.000000
2  474    1.0  3813.008333         0.0   0.000000
3  507    1.0  3838.616667         0.0   0.000000
4  484    1.0  3999.450000         0.0   0.000000
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   465    1.0  4127.725000         0.0        0.0
6   578    1.0  4260.825000         0.0        0.0
7     0    1.0  4427.816667         0.0        0.0
8   497    1.0  4561.208333         0.0        0.0
9   654    1.

Generate probs completed
[['roll_mean_1_60_nonce', 'roll_32_nonce', 'roll_std_50_60', 'roll_45', 'roll_51_nonce', 'roll_33_nonce', 'roll_46_nonce', 'roll_34_nonce', 'roll_54_nonce', 'roll_24_nonce', 'roll_22_nonce', 'roll_45_nonce', 'roll_std', 'roll_11_nonce', 'roll_5_nonce', 'roll_26', 'roll_30', 'roll_42_nonce', 'roll_std_50_60_nonce', 'roll_49_nonce', 'roll_37_nonce', 'roll_44_nonce', 'roll_56_nonce', 'roll_31_nonce', 'roll_53']]
list creation complete
******* Condition (False, 1, 20) for feature count 25 *****************
675
675
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  507    1.0  4296.897960        0.28   0.458258
1  477    1.0  4438.047602        0.24   0.435890
2  482    1.0  4200.277195        0.20   0.408248
3  568    1.0  4230.109996        0.20   0.408248
4  520    1.0  4506.737570        0.20   0.4

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,531,488,474,507,484,0.500000,0.000000,0.0,0.000000,0.000000,11.0,0,0,[]
1,5,False,1,474,654,1,507,650,0.200000,0.200000,0.2,0.200000,0.200000,7.5,1,1,[1]
2,10,False,1,520,531,465,654,560,0.300000,0.300000,0.3,0.200000,0.200000,6.5,0,1,[]
3,15,False,1,531,482,578,465,1,0.266667,0.200000,0.2,0.200000,0.200000,5.0,1,1,[1]
4,20,False,1,507,482,578,531,520,0.300000,0.200000,0.2,0.200000,0.200000,8.5,0,2,[]
5,25,False,1,507,477,482,568,520,0.280000,0.240000,0.2,0.200000,0.200000,8.0,0,2,[]
6,30,False,1,507,520,477,474,482,0.266667,0.233333,0.2,0.166667,0.166667,6.0,0,1,[]


In [111]:
%%time
prof.enable()
imp_feats_count_list = [2,5,10,15,20,25,30]
sim_cutoff_dfs=None
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']==1)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(False,1,20)])
prof.disable()
cutoff_pos_df_hash

max_good_k=457

********************** Feature count 2 *****************
Generate probs completed
[['roll_mean_1_60_nonce', 'roll_32_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 2 *****************
54
54
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  531    1.0  2638.958333         0.5   0.707107
1  488    1.0  3129.550000         0.0   0.000000
2  474    1.0  3813.008333         0.0   0.000000
3  507    1.0  3838.616667         0.0   0.000000
4  484    1.0  3999.450000         0.0   0.000000
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   465    1.0  4127.725000         0.0        0.0
6   578    1.0  4260.825000         0.0        0.0
7     0    1.0  4427.816667         0.0        0.0
8   497    1.0  4561.208333         0.0        0.0
9   654    1.

Generate probs completed
[['roll_mean_1_60_nonce', 'roll_32_nonce', 'roll_std_50_60', 'roll_45', 'roll_51_nonce', 'roll_33_nonce', 'roll_46_nonce', 'roll_34_nonce', 'roll_54_nonce', 'roll_24_nonce', 'roll_22_nonce', 'roll_45_nonce', 'roll_std', 'roll_11_nonce', 'roll_5_nonce', 'roll_26', 'roll_30', 'roll_42_nonce', 'roll_std_50_60_nonce', 'roll_49_nonce', 'roll_37_nonce', 'roll_44_nonce', 'roll_56_nonce', 'roll_31_nonce', 'roll_53']]
list creation complete
******* Condition (False, 1, 20) for feature count 25 *****************
675
675
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  507    1.0  4296.897960        0.28   0.458258
1  477    1.0  4438.047602        0.24   0.435890
2  482    1.0  4200.277195        0.20   0.408248
3  568    1.0  4230.109996        0.20   0.408248
4  520    1.0  4506.737570        0.20   0.4

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,531,488,474,507,484,0.500000,0.000000,0.0,0.000000,0.000000,11.0,0,0,[]
1,5,False,1,474,654,1,507,650,0.200000,0.200000,0.2,0.200000,0.200000,7.5,1,1,[1]
2,10,False,1,520,531,465,654,560,0.300000,0.300000,0.3,0.200000,0.200000,6.5,0,1,[]
3,15,False,1,531,482,578,465,1,0.266667,0.200000,0.2,0.200000,0.200000,5.0,1,1,[1]
4,20,False,1,507,482,578,531,520,0.300000,0.200000,0.2,0.200000,0.200000,8.5,0,2,[]
5,25,False,1,507,477,482,568,520,0.280000,0.240000,0.2,0.200000,0.200000,8.0,0,2,[]
6,30,False,1,507,520,477,474,482,0.266667,0.233333,0.2,0.166667,0.166667,6.0,0,1,[]


In [112]:
# print profiling output
stats = pstats.Stats(prof).strip_dirs().sort_stats("cumtime")
stats.print_stats(20) # top 10 rows

         797445476 function calls (797257071 primitive calls) in 284.342 seconds

   Ordered by: cumulative time
   List reduced from 1389 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.002    0.001  235.437  117.719 3966814104.py:1(create_cutoff_pos_df)
     5855    0.042    0.000  216.013    0.037 frame.py:3630(__setitem__)
     5855    0.037    0.000  215.932    0.037 frame.py:3822(_set_item)
     5855    0.039    0.000  215.802    0.037 frame.py:3797(_set_item_mgr)
     5872    0.065    0.000  214.742    0.037 generic.py:3948(_check_setitem_copy)
     5778    0.078    0.000  214.501    0.037 _exceptions.py:27(find_stack_level)
     5778    0.013    0.000  214.350    0.037 inspect.py:1552(stack)
     5778    0.332    0.000  214.333    0.037 inspect.py:1524(getouterframes)
   184896    0.899    0.000  213.855    0.001 inspect.py:1485(getframeinfo)
   231120   93.138    0.000  202.769    0.001 inspect.py:727(getmod

In [ ]:
# %%time
# imp_feats_count_list = [65,70]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']==1)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,1,20)])
# cutoff_pos_df_hash

In [ ]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs=None
# min_normal_count=50
# max_low_count = 5
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']<=max_low_count) | (df['total']>=min_normal_count)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(True,min_normal_count,10),
#                                             (False,max_low_count,20),(False,2,20)])
# cutoff_pos_df_hash

In [ ]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [ ]:
print(all_k)
print(trans_k)

In [ ]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs_actual=None
cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_actual

In [156]:
temp=df.loc[[243,181,220,226,245,10,4]]
temp['diff']=(df.loc[214,'cutoff']-temp['cutoff']).abs()
temp.sort_values('diff')

,cutoff,cutoff_2,ratio,total,diff
220,0.351447,0.352415,0.000000,2,0.028072
10,0.408559,0.409527,0.192308,26,0.029040
245,0.346607,0.347575,0.000000,2,0.032912
243,0.344671,0.345639,0.000000,5,0.034848
226,0.328215,0.329183,0.000000,0,0.051304
4,0.464704,0.465672,0.240964,83,0.085184
181,0.284655,0.285623,0.000000,0,0.094864


In [157]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs_actual=None
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(trans_k)],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_actual

In [158]:
# k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# imp_feats_count_list = [2,5,10,15,20,25]

k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
                                                      df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                            sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            imp_feats_count_list=imp_feats_count_list)
print(f'{k_sel=}')
print('cutoff best')
print(top_mean_best)
print(' ** selected test **')
test_sel[['k','client_seed','probs']]

 ........... Overall Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2     8.000000           0.666667            1.000000
1                5     5.333333           1.333333            1.666667
2               10     7.166667           1.000000            1.666667
3               15     8.166667           1.000000            1.666667
4               20     7.333333           0.666667            2.000000
5               25     8.000000           0.666667            1.333333
6               30     8.333333           0.333333            1.000000
7               40    11.000000           0.333333            1.000000
 ........... Low Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2        11.00                0.0                 0.0
1                5         7.00                0.5                 0.5
2               10         9.75               

,k,client_seed,probs
1640,189,woxpwoxpwoxpwoxpwoxpeb099c16581e2f1eb598326081e5c1232b51fe6f33091d429b5a5df7065980c9,0.322236
2477,189,woxpwoxpwoxpwoxpwoxp8857f5cc477def19a90090599758fb715f602bf79525d143de0e8e14996c0008,0.321619
3891,222,woxpwoxpwoxpwoxpwoxp957dbb2c6780b08b9cbd5c5c171bbfe28a23914406b9b5cf8da232635f1e6fa1,0.350051
5044,222,woxpwoxpwoxpwoxpwoxp5b56076a42cd0f097c7391ccc0aba2cb6f4fabdbaa4e946015ee20961732259f,0.350428
4154,229,woxpwoxpwoxpwoxpwoxp2d55940bdd307fe9432ce8e5fc422c22704b3c1bc2cc1f5e53d891be95d1708f,0.331125
7326,229,woxpwoxpwoxpwoxpwoxp90643b7aee4cbf42ebf9f383fd48c51ffccd4ca4e31738ef3aa13d9ce33c9ed6,0.331267
1145,241,woxpwoxpwoxpwoxpwoxp5afa5df5cad54ce8a550d8808faf3f992a16ecc30f40c72cf4a8c9a456475f0a,0.343361
3046,241,woxpwoxpwoxpwoxpwoxp1cd2c38f9bcfafaab285b27412e43399bbb1b6fc72854a4beb83215421bc6812,0.343157


In [468]:
test_sel=None
for k_sel in [455,508,432,460,469,515]:
    row = df.iloc[k_sel]
#     print(row)
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    cur_test_sel = test_probs[mask]
    cur_test_sel['k']=k_sel
    if test_sel is None:
        test_sel = cur_test_sel
    else:
        test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
    print('Test Size:',len(test_sel))
test_sel

Test Size: 1
Test Size: 2
Test Size: 3
Test Size: 4
Test Size: 5
Test Size: 6


,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [172]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

,k,roll_mean_1_60,roll_42_nonce,roll_60,roll_19_nonce,roll_mean_nonce,roll_mean,roll_mean_25_50_nonce,roll_mean_1_60_nonce,roll_7_nonce,roll_32_nonce
2012,130,4302.066667,9624,9386,5737,5313.9,3959.95,5269.384615,4814.133333,8607,7325
4683,130,5999.000000,1275,9665,7312,6164.5,5416.00,5277.230769,5704.983333,7651,6917
2657,146,4368.300000,9696,9990,7769,5958.4,3656.20,5253.230769,5364.416667,3299,6928
2983,146,4582.866667,7868,9864,6686,5012.5,4039.75,4458.269231,5040.666667,301,4690


In [175]:
sim_hash_group[sim_hash_group['k']==24]

,k,hash_group,ratio_mean,ratio_std
5,24,0.0,0.130841,0.0


In [176]:
sim_cutoff_df[sim_cutoff_df['k']==24]

,k,ratio,total,hash_no,hash_group
48,24,0.130841,214,roll_21_nonce,0.0
49,24,0.130841,214,roll_37_nonce,0.0


In [586]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [587]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [588]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())


quantile: 0.01
Actuals: 0.05 1 20
Hash: 0.16666666666666666 2 12
12 20

quantile: 0.02918367346938775
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.04836734693877551
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.06755102040816326
Actuals: 0.058823529411764705 1 17
Hash: 0.1 1 10
10 17

quantile: 0.086734693877551
Actuals: 0.058823529411764705 1 17
Hash: 0.1111111111111111 1 9
9 17

quantile: 0.10591836734693875
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.12510204081632653
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.14428571428571427
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.16346938775510203
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.1826530612244898
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.20183673469387753
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 16

quantile: 0.2210204081632653
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 

In [589]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

0.05 1 20


In [559]:
test_sel = best_actual.copy()

In [261]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [590]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

0 0 0 0
1 0.0 0 1
2 0.0 0 2
3 0.0 0 3
4 0.0 0 5
5 0.0 0 5
6 0.0 0 6
7 0.0 0 8
8 0.0 0 8
9 0.0 0 9
10 0.08333333333333333 1 12


In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [166]:
def gen_analysis(nonce,train_manual,test_manual,test,models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",save_files=True,gen_probs=True,
                analysis_folder="analysis"):
    if CONT_ANALYSIS:
        analysis_df=pd.read_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv')
    else:
        analysis_df=None
    print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
    for zero_cutoff in range(0,6):
        analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
                            analysis_df=analysis_df)
    print('Train completed')
    analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
    print('Test completed')   
    
    if save_files:
        joblib.dump(xgb_models,f'data/models/models_{nonce}{file_suffix}.dump')
        print('Models Save completed')   

    features_test = [col for col in features if col not in ['target','probs']]
    
    if gen_probs:
        test=generate_probs(True,models,test,features_test)
    else:
        test['probs'] = np.load(f'data/probs/probs_{nonce}{file_suffix}.npy')

    if save_files:
        np.save(f'data/probs/probs_{nonce}{file_suffix}.npy',test['probs'].values)
    print('Probability Prediction completed')   

    test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
                                                models,
                                               np.mean(np.array(tr_last_cutoffs)),
                                                 np.mean(np.array(val_cutoffs)), 
                                                PATTERN_99,HIGH_ANALYSIS,multi_models=multi_models,
                                                  gen_probs=False)
    print(len(analysis_df))
    analysis_df.to_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv',index=False)  
    print('Analysis Generation completed')  
    return analysis_df,test

In [131]:
test['roll_actual'].head()

0    1119
1    1638
2    2244
3    5933
4    3730
Name: roll_actual, dtype: int64

In [167]:
%%time
analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",
                save_files=True,analysis_folder="analysis_25_to_50_seeds"
#                 gen_probs=False
                    )
analysis_df.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:4 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:4 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:6 cutoff:5 after_val:8
Function time:7.605552673339844
972
Analysis Generation completed
CPU times: user 14.8 s, sys: 168 ms, total: 14.9 s
Wall time: 12 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,13.636364,15.000000,12.345679,12.345679,12.345679,12.000000,11.111111,11.538462,12.500000,0.0,0.0,0.0,0.0,81.0,81.0,81.0,75.0,18.0,44.0,40.0,78.0,80.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,8.333333,7.692308,11.538462,11.538462,11.538462,9.090909,0.000000,8.695652,12.000000,0.0,0.0,0.0,0.0,26.0,26.0,26.0,22.0,3.0,12.0,13.0,23.0,25.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,6.896552,7.407407,10.909091,10.909091,10.909091,10.000000,0.000000,9.615385,11.111111,0.0,0.0,0.0,0.0,55.0,55.0,55.0,50.0,9.0,29.0,27.0,52.0,54.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,11.363636,11.965812,11.489362,11.489362,11.489362,10.810811,2.380952,10.869565,11.538462,0.0,0.0,0.0,0.0,235.0,235.0,235.0,222.0,42.0,132.0,117.0,230.0,234.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,17.857143,20.408163,17.171717,17.171717,17.171717,17.391304,0.000000,16.666667,17.346939,0.0,0.0,0.0,0.0,99.0,99.0,99.0,92.0,10.0,56.0,49.0,96.0,98.0


In [168]:
%%time
analysis_df_hash,test_hash = gen_analysis(nonce,train_manual,test_manual_hash,test_hash,
                                xgb_models,tr_last_cutoffs,val_cutoffs,
                                multi_models=True,file_suffix="_hash",
                                save_files=True,
                                         )
analysis_df_hash.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
Function time:6.502852916717529
972
Analysis Generation completed
CPU times: user 13.8 s, sys: 204 ms, total: 14 s
Wall time: 10.9 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,9.615385,8.695652,7.608696,7.608696,7.608696,7.954545,0.000000,7.777778,7.692308,0.0,0.0,0.0,0.0,92.0,92.0,92.0,88.0,20.0,52.0,46.0,90.0,91.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,11.111111,12.500000,9.375000,9.375000,9.375000,9.375000,0.000000,9.375000,9.375000,0.0,0.0,0.0,0.0,32.0,32.0,32.0,32.0,6.0,18.0,16.0,32.0,32.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,5.882353,6.666667,6.666667,6.666667,6.666667,6.779661,0.000000,6.779661,6.779661,0.0,0.0,0.0,0.0,60.0,60.0,60.0,59.0,13.0,34.0,30.0,59.0,59.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,10.156250,7.964602,10.176991,10.176991,10.176991,10.628019,10.526316,10.138249,10.360360,0.0,0.0,0.0,0.0,226.0,226.0,226.0,207.0,38.0,128.0,113.0,217.0,222.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,5.882353,6.818182,5.617978,5.617978,5.617978,6.097561,14.285714,5.813953,5.747126,0.0,0.0,0.0,0.0,89.0,89.0,89.0,82.0,14.0,51.0,44.0,86.0,87.0


In [ ]:
# %%time
# if CONT_ANALYSIS:
#     analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')
# else:
#     analysis_df=None
# print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
# for zero_cutoff in range(0,6):
#     analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=analysis_df)
# print('Train completed')
# analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=True)
# print('Test completed')

In [ ]:
# %%time

# # model_no=5
# # analysis_df = save_test_ml_probs(analysis_df,test,
# #                                 xgb_models[model_no],np.mean(np.array(tr_last_cutoffs)),
# #                                 np.mean(np.array(val_cutoffs)))


# #Save the Models
# joblib.dump(xgb_models,f'models_{nonce}.dump')

# features_test = [col for col in features if col not in ['target','probs']
# test=generate_probs(True,xgb_models,test,features_test)
# np.save(f'probs/probs_{nonce}.npy',test['probs'].values)
                 
# test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
#                                             xgb_models,
#                                            np.mean(np.array(tr_last_cutoffs)),
#                                              np.mean(np.array(val_cutoffs)), 
#                                             PATTERN_99,HIGH_ANALYSIS,multi_models=True,
#                                               gen_probs=False)
# print(len(analysis_df))
# analysis_df.to_csv(f'data/analysis/analysis_df_{nonce}.csv',index=False)

In [ ]:
# xgb_models= joblib.load(f'data/models/models_{nonce}.dump')
# for model in xgb_models:
#     temp_probs = model.predict_proba(test[0:5][features_test])
#     print(temp_probs)

In [ ]:
# len(analysis_df[(analysis_df['pattern_99']==True) & (analysis_df['target_high']==False)])

In [ ]:
# analysis_df=pd.read_csv(f'data/analysis/analysis_df_2582.csv')

In [169]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

In [ ]:
# %%time

# suffix='_hash'
# digit_list=[1,2,3,4]
# # digit_list=[9,8,7,5]
# zero_col_cutoff_list=[0,1]

# cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#        'pr_tr_cutoff','pr_val_cutoff']
# analysis_list_hash=[]
# mean_list=[]
# model_means=[]
# mask = analysis_df['digit'].isin(digit_list)
# mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
# cutoffs = [(13,20),(13,50),(13,100),(14,20),
#            (14,100),(14,50),(15,20),(15,50),(15,100)]
# cutoffs =[(18,3)]

# # if CONT_ANALYSIS:
# #     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# # else:
# #     analysis_mean_df = None

# analysis_mean_df_hash =None    
# mean_df_hash=None

# for (ratio_cutoff,count_cutoff) in cutoffs:

#     filtered = report_exceptional_teratio(analysis_df_hash,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                          )
#     print(len(filtered))
# #     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
# #                                     xgb_models,
# #                                    np.mean(np.array(tr_last_cutoffs)),
# #                                      np.mean(np.array(val_cutoffs)), 
# #                                   PATTERN_99,HIGH_ANALYSIS,
# #                                   gen_probs=False,
# #                                   multi_models=True )
#     analysis_list_hash.append(filtered)
#     records  = len(filtered)
# #     print('Records:',records)
# #     cur_mean = filtered[cols].mean(axis=0)
#     cur_mean = pd.Series()
#     pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                        'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                       'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                      'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                      'te_ratio':'te_total'}
    
#     for pr_col,pr_count_col in pr_count_dict.items():
#         cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
#     model_mean = cur_mean.mean()
#     te_total_sum = filtered['te_total'].sum(axis=0)
# #     model_mean = filtered[cols].mean(axis=None)
# #     print('Overall mean:',model_mean)
# #     print(cur_mean)

#     cur_mean.loc['overall_mean']=model_mean
#     cur_mean.loc['ratio_cutoff']=ratio_cutoff
#     cur_mean.loc['total_cutoff']=count_cutoff
#     cur_mean.loc['records']=records
#     cur_mean.loc['te_total_sum']=te_total_sum

#     if mean_df_hash is None:
#         mean_df_hash = pd.DataFrame({'temp':cur_mean})    
#     else:
#         mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

# mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
# mean_df_hash['nonce']=nonce
# mean_df_hash['pattern_99']=PATTERN_99
# mean_df_hash['target_high']=HIGH_ANALYSIS
# key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#             'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
#             'overall_mean']
# cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

# mean_df_hash=mean_df_hash[cols_reorder]
# # mean_df
# # temp_df
# if analysis_mean_df_hash is None:
#     analysis_mean_df_hash=mean_df_hash 
# else:
#     analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
# analysis_mean_df_hash

In [ ]:
# %%time

# if not IS_PROD:

#     cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#            'pr_tr_cutoff','pr_val_cutoff']
#     analysis_list=[]
#     mean_list=[]
#     model_means=[]
#     mask = analysis_df['digit'].isin(digit_list)
#     mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
#     #            (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
#     #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
#     # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(18,3)]

# #     if CONT_ANALYSIS:
# #         analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
# #     else:
# #         analysis_mean_df = None

#     analysis_mean_df =None    
#     mean_df=None

#     for (ratio_cutoff,count_cutoff) in cutoffs:

#         filtered = report_exceptional_teratio(analysis_df,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                              )
   
    
# #         filtered = analysis_df[mask1]
#         print(len(filtered))
# #         test_filt_all,filtered = save_test_ml_probs(filtered,test,
# #                                         xgb_models,
# #                                        np.mean(np.array(tr_last_cutoffs)),
# #                                          np.mean(np.array(val_cutoffs)), 
# #                                       PATTERN_99,HIGH_ANALYSIS,
# #                                       gen_probs=False,
# #                                       multi_models=True )
#         analysis_list.append(filtered)
#         records  = len(filtered)
#     #     print('Records:',records)
# #         cur_mean = filtered[cols].mean(axis=0)
#         cur_mean = pd.Series()
#         pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                            'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                           'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                          'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                          'te_ratio':'te_total'}

#         for pr_col,pr_count_col in pr_count_dict.items():
#             cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
#         model_mean = cur_mean.mean()
#         te_total_sum = filtered['te_total'].sum(axis=0)
        
        
#     #     model_mean = filtered[cols].mean(axis=None)
#     #     print('Overall mean:',model_mean)
#     #     print(cur_mean)

#         cur_mean.loc['te_total_sum']=te_total_sum
#         cur_mean.loc['overall_mean']=model_mean
#         cur_mean.loc['ratio_cutoff']=ratio_cutoff
#         cur_mean.loc['total_cutoff']=count_cutoff
#         cur_mean.loc['records']=records

#         if mean_df is None:
#             mean_df = pd.DataFrame({'temp':cur_mean})    
#         else:
#             mean_df=pd.concat([mean_df,cur_mean],axis=1)

#     mean_df=mean_df.transpose().reset_index(drop=True)
#     mean_df['nonce']=nonce
#     mean_df['pattern_99']=PATTERN_99
#     mean_df['target_high']=HIGH_ANALYSIS
#     key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#                 'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
#                 'pr_mean','pr_med','overall_mean']
#     cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

#     mean_df=mean_df[cols_reorder]
#     # mean_df
#     # temp_df
#     if analysis_mean_df is None:
#         analysis_mean_df=mean_df 
#     else:
#         analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
# analysis_mean_df

In [171]:
%%time

suffix='_hash'
digit_list=[1,2,3,4]
digit_list=[9,8,7,5]
# digit_list=[6]
zero_col_cutoff_list=[0,1]
only_better_probs = False
better_prob_cutoff= 11
cutoff_col = 'te_ratio'


cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
       'pr_tr_cutoff','pr_val_cutoff']
analysis_list_hash=[]
mean_list=[]
model_means=[]
mask = analysis_df['digit'].isin(digit_list)
mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
cutoffs = [(13,15),(13,20),(13,50),(13,100),(14,15),(14,20),
           (14,100),(14,50),(15,15),(15,20),(15,50),(15,100),(18,50),(20,50),(25,50),]
# cutoffs =[(14,20)]

# if CONT_ANALYSIS:
#     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# else:
#     analysis_mean_df = None

analysis_mean_df_hash =None    
mean_df_hash=None

for (ratio_cutoff,count_cutoff) in cutoffs:

    mask1= mask & (analysis_df_hash['tr_ratio']>=ratio_cutoff) \
                & (analysis_df_hash['te_total']>=count_cutoff) \
                & (analysis_df_hash['pattern_99']==PATTERN_99) \
                & (analysis_df_hash['target_high']==HIGH_ANALYSIS) \
    
    if only_better_probs:
        mask1 = mask1 & (analysis_df_hash[cutoff_col]>=better_prob_cutoff)

    filtered = analysis_df_hash[mask1]
    print(len(filtered))
#     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
#                                     xgb_models,
#                                    np.mean(np.array(tr_last_cutoffs)),
#                                      np.mean(np.array(val_cutoffs)), 
#                                   PATTERN_99,HIGH_ANALYSIS,
#                                   gen_probs=False,
#                                   multi_models=True )
    analysis_list_hash.append(filtered)
    records  = len(filtered)
#     print('Records:',records)
#     cur_mean = filtered[cols].mean(axis=0)
    cur_mean = pd.Series()
    pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                       'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                      'pr_mean':'te_mean_count','pr_med':'te_med_count',
                     'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                     'te_ratio':'te_total'}
    
    for pr_col,pr_count_col in pr_count_dict.items():
        cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
    model_mean = cur_mean.mean()
    te_total_sum = filtered['te_total'].sum(axis=0)
#     model_mean = filtered[cols].mean(axis=None)
#     print('Overall mean:',model_mean)
#     print(cur_mean)

    cur_mean.loc['overall_mean']=model_mean
    cur_mean.loc['ratio_cutoff']=ratio_cutoff
    cur_mean.loc['total_cutoff']=count_cutoff
    cur_mean.loc['records']=records
    cur_mean.loc['te_total_sum']=te_total_sum

    if mean_df_hash is None:
        mean_df_hash = pd.DataFrame({'temp':cur_mean})    
    else:
        mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
mean_df_hash['nonce']=nonce
mean_df_hash['pattern_99']=PATTERN_99
mean_df_hash['target_high']=HIGH_ANALYSIS
key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
            'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
            'overall_mean']
cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

mean_df_hash=mean_df_hash[cols_reorder]
# mean_df
# temp_df
if analysis_mean_df_hash is None:
    analysis_mean_df_hash=mean_df_hash 
else:
    analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
analysis_mean_df_hash

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
CPU times: user 304 ms, sys: 15 µs, total: 304 ms
Wall time: 303 ms


,nonce,pattern_99,target_high,ratio_cutoff,total_cutoff,records,te_total_sum,te_ratio,pr_tr_cutoff,pr_val_cutoff,pr_mean,pr_med,overall_mean,pr_1,pr_2,pr_3,pr_4,pr_5
0,2649,False,False,13.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2649,False,False,13.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2649,False,False,13.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2649,False,False,13.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2649,False,False,14.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2649,False,False,14.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2649,False,False,14.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2649,False,False,14.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2649,False,False,15.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2649,False,False,15.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
analysis_list_hash[0]

In [144]:
print(len(analysis_df_hash))
len(analysis_df_hash[analysis_df_hash['te_ratio']!=0])

972


97

In [145]:
print(len(analysis_df))
len(analysis_df[analysis_df['te_ratio']!=0])

972


95

In [ ]:
%%time

if not False: #IS_PROD:

    cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
           'pr_tr_cutoff','pr_val_cutoff']
    analysis_list=[]
    mean_list=[]
    model_means=[]
    mask = analysis_df['digit'].isin(digit_list)
    mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
    #            (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
    #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
    # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(14,20)]

    if CONT_ANALYSIS:
        analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
    else:
        analysis_mean_df = None

    analysis_mean_df =None    
    mean_df=None

    for i,(ratio_cutoff,count_cutoff) in enumerate(cutoffs):

    #     mask1= mask & (analysis_df['cutoff']==5) \
    #                 & (analysis_df['te_total']>=count_cutoff) \
    #                 & (analysis_df['tr_ratio']>=ratio_cutoff) \
    #                 & (analysis_df['zero_col_cutoff']==3) \
        mask1= mask & (analysis_df['tr_ratio']>=ratio_cutoff) \
                    & (analysis_df['te_total']<=count_cutoff) \
                    & (analysis_df['pattern_99']==PATTERN_99) \
                    & (analysis_df['target_high']==HIGH_ANALYSIS) \
    #                  & (analysis_df['zero_col_cutoff'].isin([0,1])) \
    #                  & (analysis_df['cutoff'].isin([4,5])) \
       #                 & ((analysis_df['aft_all']>13) | (analysis_df['aft_1']>13) | (analysis_df['aft_2']>13)) \
        #                 & ((analysis_df['aft_all']<13) | (analysis_df['aft_1']<13) | (analysis_df['aft_2']<13)) \
        #                  & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

        if only_better_probs:
    #         mask1 = mask1 & (analysis_df.index.isin(analysis_list_hash[i].index))
            filtered=analysis_df.loc[analysis_list_hash[i].index][mask1]
        else:
            filtered = analysis_df[mask1]
            
        print(len(filtered))
        test_filt_all,filtered = save_test_ml_probs(filtered,test,
                                        xgb_models,
                                       np.mean(np.array(tr_last_cutoffs)),
                                         np.mean(np.array(val_cutoffs)), 
                                      PATTERN_99,HIGH_ANALYSIS,
                                      gen_probs=False,
                                      multi_models=True )
        analysis_list.append(filtered)
        records  = len(filtered)
    #     print('Records:',records)
#         cur_mean = filtered[cols].mean(axis=0)
        cur_mean = pd.Series()
        pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                           'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                          'pr_mean':'te_mean_count','pr_med':'te_med_count',
                         'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                         'te_ratio':'te_total'}

        for pr_col,pr_count_col in pr_count_dict.items():
            cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
        model_mean = cur_mean.mean()
        te_total_sum = filtered['te_total'].sum(axis=0)
        
        
    #     model_mean = filtered[cols].mean(axis=None)
    #     print('Overall mean:',model_mean)
    #     print(cur_mean)

        cur_mean.loc['te_total_sum']=te_total_sum
        cur_mean.loc['overall_mean']=model_mean
        cur_mean.loc['ratio_cutoff']=ratio_cutoff
        cur_mean.loc['total_cutoff']=count_cutoff
        cur_mean.loc['records']=records

        if mean_df is None:
            mean_df = pd.DataFrame({'temp':cur_mean})    
        else:
            mean_df=pd.concat([mean_df,cur_mean],axis=1)

    mean_df=mean_df.transpose().reset_index(drop=True)
    mean_df['nonce']=nonce
    mean_df['pattern_99']=PATTERN_99
    mean_df['target_high']=HIGH_ANALYSIS
    key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
                'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
                'pr_mean','pr_med','overall_mean']
    cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

    mean_df=mean_df[cols_reorder]
    # mean_df
    # temp_df
    if analysis_mean_df is None:
        analysis_mean_df=mean_df 
    else:
        analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
    analysis_mean_df

In [ ]:
analysis_mean_df

In [ ]:
analysis_list[0]

In [ ]:
print(test_hash['probs'].describe())
cutoff = test_hash['probs'].quantile(q=0.97)
print(cutoff)
mask = test_hash['probs']>cutoff
total = len(test_hash[mask])
mask2 = test_hash['roll_actual']>=8944
success = len(test_hash[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
print(test['probs'].describe())
cutoff = test['probs'].quantile(q=0.97)
print(cutoff)
mask = test['probs']>cutoff
total = len(test[mask])
mask2 = test['roll_actual']>=8944
success = len(test[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
if not IS_PROD:
    analysis_mean_df.to_csv(f'data/analysis/analysis_mean_df_{nonce}.csv',index=False)

##### Production Code

In [ ]:
# digitlist=[9,8,7,5]
# zerocolcutoffs=list(range(0,6))

# digitlist=[1,2,3,4]
digitlist=[9,8,7,5]
zerocolcutoffs=[4,5]
ratio_cutoff=15
count_cutoff_min=20
count_cutoff_max=10000
analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
anal_size=len(analysis_filt)
test_size=analysis_filt['te_total'].sum()
print(f'Analysis Selection Size: {anal_size} Total Test Size:{test_size}')

In [ ]:
analysis_filt

In [ ]:
#Fetch test selection based on te ratio 

def get_test_from_analysis(analysis_df,test,
              pattern,bHighCheck,
              is_prod=False,):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
        print(f'{filt_size=}')
        mask=create_target_mask(test_filt_df)
        success = len(test_filt_df[mask])
#         success=len(test_filt_df[test_filt_df['roll_actual']>HIGH_TARGET])
        print(f'{success=}')        
        
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue

        if is_prod:
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all

In [ ]:
# temp_sel = save_test_stats(analysis_filt, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
# print(len(temp_sel))

In [ ]:
# %%time
# test_sel = get_test_from_analysis(analysis_filt,test,PATTERN_99,HIGH_ANALYSIS,
#                                   is_prod=True)
# print(len(test_sel))

In [ ]:
%%time
test_sel,ret_analysis_df = save_test_ml_probs(analysis_filt,
                                              test,
                                    xgb_models,
                                   np.mean(np.array(tr_last_cutoffs)),
                                  np.mean(np.array(val_cutoffs)), 
                                  PATTERN_99,HIGH_ANALYSIS,
                                  is_prod=True,
                                 gen_probs=False,
                                  prob_cutoff=0.3,
                                  multi_models=True )
print(f'Selected Test Size:{len(test_sel)}')

In [ ]:
if not IS_PROD:
    success=len(test_sel[test_sel['roll_actual']>=9000])
    print(success)
    print(success / len(test_sel))

In [ ]:
test_sel.head()

In [469]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [470]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

rand_pos:5
Client Seed Selected
woxpwoxpwoxpwoxpwoxpcf0e00e2b60b297d6b6dcb153888a52b070849ac798b0ac10480586d5bdae7eb


In [471]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [147]:
final_test.reset_index()[final_test.reset_index()['index']==6826]

,index,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actu

After Actual SEED is known

In [472]:
actual_seed="b0ff658797d080520bc6d528365e73a5fa1aa72b52c6f3069a32b0a595ef1263"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [473]:
cur_hash_list[25]

'7b4f7fc740f40bce27a72b8d28e42321f8bc26314f1d9a1bc6dd58a022f1c191'

In [474]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [475]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actual_30,roll

In [287]:
final_test.to_csv(f'data/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [162]:
LOW_TARGET

1000

In [159]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

1
success_pos:Int64Index([6865], dtype='int64')


In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

In [ ]:
# ratio_cutoff=18
# count_cutoff_min=3
# # test_sel_options= [([1],[0,1],10000),
# #                     ([2],[0,1],10000),
# #                    ([3],[0,1],10000),
# #                    ([4],[0,1],10000),
# #                     ([5],[0,1],10000),
# # #                     ([6],[0,1],10000),
# #                    ([7],[0,1],10000),
# #                    ([8],[0,1],10000),
# #                    ([9],[0,1],10000),
# #                     ([1],[4,5],10000),
# #                     ([2],[4,5],10000),
# #                    ([3],[4,5],10000),
# #                    ([4],[4,5],10000),
# #                     ([5],[4,5],10000),
# # #                     ([6],[4,5],10000),
# #                    ([7],[4,5],10000),
# #                    ([8],[4,5],10000),
# #                    ([9],[4,5],10000),
# #                   ]

# test_sel_options= [([1,2,3,4],[0,1],10000),
#                     ([9,8,7,5],[0,1],10000),
#                    ([6],[0,1],10000),
#                     ([1,2,3,4],[4,5],10000),
#                     ([9,8,7,5],[4,5],10000),
#                    ([6],[4,5],10000),
#                   ]
# test_sel_options= [([1],[0],10000),]

#                    #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),

# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]


# # test_sel_options= [([9,8,7,6,5,1,2,3,4],[0,1,4,5],10000)]
# # test_sel_options= [([9,8,7,5],[4,5],10000)]
# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]

# anal_sizes =[]
# analysis_filts=[]
# test_sizes=[]
# filt_options=[]
# for option in test_sel_options:
#     digitlist=option[0]
#     zerocolcutoffs=option[1]
#     count_cutoff_max=option[2]
#     analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
#                          ratio_cutoff,count_cutoff_min,count_cutoff_max,
#                          PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
#     anal_size=len(analysis_filt)
#     if anal_size!=0:
#         test_size=analysis_filt['te_total'].sum()
#         print(f'option: {option} Analysis Selection Size: {anal_size} Total Test Size:{test_size}')
#         break
# #         anal_sizes.append(anal_size)
# #         analysis_filts.append(analysis_filt)
# #         filt_options.append(option)
# #         test_sizes.append(test_size)
#     else:
#         print(f'No records for option:{option}')

# # sel_option = np.argmin(np.array(test_sizes))
# # print(f'selected option: {filt_options[sel_option]} \
# #       Analysis Selection Size: {anal_sizes[sel_option]} Total Test Size:{test_sizes[sel_option]}')

# # analysis_filt = analysis_filts[sel_option]

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')